In [26]:
import os
import sys
sys.path.insert(0, '/home/comp/cscxliu/COAT') 

from utils import Logger, get_value_from_responce
from openai_utils import LLM_openai


import numpy as np
import pandas as pd
from causallearn.search.ConstraintBased.FCI import fci
from causallearn.utils.GraphUtils import GraphUtils
from copy import  deepcopy
from scipy import stats
from causallearn.utils.cit import CIT
from sklearn.cluster import KMeans

# load groundtruth 
meta = pd.read_excel('Apple_Gastronome_AG7_v20240201.xlsx')

# san check for groundtruth
values = meta.values[:,:-1].astype(float)
names = list(meta.columns[:-1])

meta.head()

,score,color,size,smell,taste,juiciness,recmd,Review
0,-2,1,0,-1,-1,-1,-1,"Despite a visually appealing and vibrant hue,..."
1,-1,1,1,-1,-1,1,-1,"Despite its brilliant hue and grandeur, this ..."
2,2,1,1,1,1,1,1,"This apple's vibrant, consistent hue and larg..."
3,0,1,-1,1,-1,-1,1,"With a vibrant, uniform color, this pint-size..."
4,-1,1,-1,-1,1,1,0,"Despite its vibrant, uniform hue, the apple's..."


In [28]:

# Causal Discovery (FCI)
g, edges = fci(values, alpha = 0.05, verbose=False)

# visualization
pdy = GraphUtils.to_pydot(g, labels=names)
print(pdy.to_string())

Depth=0, working on node 6: 100%|██████████| 7/7 [00:00<00:00, 877.68it/s]

X1 --> X7
digraph  {
dpi=200;
fontsize=18;
0 [label=X1];
0 [label=score];
1 [label=X2];
1 [label=color];
2 [label=X3];
2 [label=size];
3 [label=X4];
3 [label=smell];
4 [label=X5];
4 [label=taste];
5 [label=X6];
5 [label=juiciness];
6 [label=X7];
6 [label=recmd];
2 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
3 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
4 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
0 -> 6  [arrowhead=normal, arrowtail=none, dir=both];
1 -> 6  [arrowhead=normal, arrowtail=odot, dir=both];
4 -> 5  [arrowhead=odot, arrowtail=odot, dir=both];
}



In [29]:
import fastapi_poe as fp
import asyncio

async def chat(content, bot_name="GPT-3.5-Turbo" ):
    poe_api_key = "your api key"
    message = fp.ProtocolMessage(role="user", content=content)
    responce = ""
    async for partial in fp.get_bot_response(messages=[message], bot_name=bot_name, api_key=poe_api_key): 
        responce = responce + partial.text

    return responce

In [30]:
await chat("Hello world!", bot_name="GPT-3.5-Turbo" )

'Hello there! How can I assist you today?'

# Utils

In [31]:
def GetMB(G, node_name):
    mbset = []
    d = G.shape[0]
    pa = lambda x: [idx for idx in range(d) if G[idx, x] not in [1, 0]]
    ch = lambda x: [idx for idx in range(d) if G[x, idx] not in [1, 0]]
    mbset += ch(0)
    ch_pa = []
    for each_ch in mbset:
        ch_pa += pa(each_ch)
    mbset += ch_pa + pa(0)

    return np.array(node_name)[list(set(mbset))]

In [32]:
def MBSep(x,y,p):
    return [i for i in range(p) if i not in [x,y]]

In [33]:
def get_factor_name(this_factor):

    this_factor = this_factor.split('\n')[0].lower()

    for i in range(20):
        if f'{i}.' in this_factor:
            this_factor = this_factor.replace(f'{i}.', '').strip()
    
    for s in ":":
        if s in this_factor:
            this_factor = this_factor.replace(s, '').strip()

    return this_factor

In [34]:
def check_factor_list(factor_list):
    factor_list = [f for f in factor_list if len(f)>10]
    factor_list = [f for f in factor_list if (" 1:" in f)and((" -1:" in f)and(" 0:" in f))]

    for outer_idx, each_str in enumerate(factor_list):
        for idx, ch in enumerate(each_str):
            if ch.isalpha():
                break 
        each_str_list = each_str[idx:].split('\n')

        each_str = each_str_list[0].lower()
        each_str_2 = '\n'.join(each_str_list[1:])

        for i in range(100):
            if f'{i}' in each_str:
                each_str = each_str.replace(f'{i}', '').strip()
            
        
        for s in ['factor', "*", 'name', ":", '.']:
            if s in each_str:
                each_str = each_str.replace(s, '').strip()
                
        if len(each_str)>0:
            factor_list[outer_idx] = each_str+'\n'+each_str_2
        else:
            factor_list[outer_idx] = ''
    
    factor_list = [f for f in factor_list if len(f)>10]

    return factor_list

In [35]:
names

['score', 'color', 'size', 'smell', 'taste', 'juiciness', 'recmd']

In [36]:
data = ''
for g in np.unique(values[:,0]):
    data += f'\n## Group with \'Score\' = {int(g)}\n\n'
    g_indeces = np.arange(len(values[:,0]))[values[:,0]==g]
    g_indeces = np.random.choice(g_indeces, min(len(g_indeces), 3), replace=False)
    for i in g_indeces:
        this_review = meta.values[i,-1].replace('\n','').strip()
        data += f"- {this_review}\n"
print(data)


## Group with 'Score' = -4

- This particular apple, while boasting a striking and consistent hue, falls short of expectations due to its unappealing small stature, an off-putting musty odor suggesting decay, and a flavor profile that tilts disappointingly toward sourness. The lack of juiciness further detracts from its desirability, making it a risky choice for suppliers and a potential loss-inducer in the market.
- Unfortunately, this batch of apples fails to meet quality standards. Their small size, extensive surface blemishes, and dry texture fall short of expectations. The off-putting musty scent suggests potential rot, and the sour taste overpowers any sweetness. Suppliers should beware; these apples could result in financial disappointment if offered to discerning customers.

## Group with 'Score' = -3

- This apple's vibrant, uniform color is its sole appealing feature. The small size, musty odor, and unpalatable sourness far outweigh initial visual impressions, while the parc

In [37]:
annotation_prompt = lambda factor_state, text : f''' 

You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data

The reviewer comments on an apple that are randomly picked:
{text} 


# Tasks 

For the given sample, what is the most approciate facotr value based on the criterion?

{factor_state}


**Your final output should follow this template**:
**"The value is: ___."**

## Output

### Part 1. analysis

In this part, feel free to write down the process of your considerations. 

Hint
- You need to abstract, identify, and choose suitable values. You may write down your hypothesis.
-  each value has one specific criterion, you should choose the most suitable one.
- **Your final output should follow this template**:
    **"The value is: ___."**

direct copy helpful information from sample that related to factor criterion, if any.

### Part 4. Final output

In this part, you are required to report the value

**Your final output should follow this template**:
**"The value is: ___."**

'''

In [38]:

def propose_prompt_with_data_in_front(example, used_factors = None):
    if used_factors is None:
        used_factors_hint = ''
    else:
        used_factors_hint = "\n - Avoid overlapping with **those existing factors:**\n\t- " + '\n\t- '.join(used_factors) + '\n'
    prompt = f'''
You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data

{example}

# Tasks: Factor Abstraction.

**What are the high-level factors behind the text that contribute to the allocation of groups?**

- Propose your candidate factors based on your observation on given data.
- The proposed factors should be diverse and different senmatically. Their overlapping should be minimized.

# About Output

Your output should contain parts described as follows.

**Part 1**: Consideration.

In this part, feel free to write down the process of your considerations. 
Hint: You need to abstract, identify, and design suitable factors with coresponding criteria, and each factor is only allowed to take value from  [-1, 0, 1] .  

**Part 2**: Factor filtration. You shoud decide whether to use each of the proposed factors by following criteria:
- Data availability. The concrete value of each new factor could be easily and clearly read off from most of samples in given data. If this point is not satisfied, you should not use this factor.
- Informative. Each new factor should exhibit relative stability within its respective group while manifesting significant variation across the various groups. If this point is not satisfied, you should not use this factor.
- Each Factor should focus on one concrete aspect and try to avoid overlape. If this point is not satisfied, you should not use this factor. {used_factors_hint}

**Part 3**: Final Output. 

 In this part, you are required to report the factors that you decided to use. 
- Each factors' value should from **[-1, 0, 1]** 
- **You should give specific criterion for each value of each factor.**
- Keep the criterion for value 0 fixed as "Otherwise; or not mentioned". 


Report the factors **in following template:**

```
**Factor Name**
- 1: [Positive Criterion]
- 0: [Otherwise; or not mentioned].
- -1: [Negative Criterion]
```

'''
    return prompt

In [39]:
annotation = {}

# Ground truth

In [40]:
def get_possible_ancestors(G, annotated_name):
    d = G.shape[0]

    # A[i,j] = 1 iff there is possibly a edge from j to i
    A = np.zeros((d,d))
    for i in range(d):
        for j in range(d):
            if i==j:
                continue
            if (G[i,j]==-1) and (G[j,i]==1): # i --> j
                A[j,i] = 1

            if (G[i,j]==2) and (G[j,i]==1): # i o-> j
                A[j,i] = 1

            if (G[i,j]==2) and (G[j,i]==2): # i o-o j
                A[j,i] = 1
                A[i,j] = 1

            if (G[i,j]==1) and (G[j,i]==1): # i <-> j
                pass 

    possible_path = A>0
    for _ in range(d):
        possible_path |= (possible_path@A)>0

    return [annotated_name[idx] for idx in range(d) if possible_path[0,idx]]

In [41]:
gt_node_names = [
    'Score', # 0
    'Size',  # 1
    'Aroma', # 2
    'Taste', # 3
    'Color', # 4
    'recmd', # 5
    'Juicy', # 6
]

gt_A = np.zeros((7,7), int)

gt_A[0, :] = [0, 1, 1, 1, 0, 0, 0]
gt_A[5, :] = [1, 0, 0, 0, 1, 0, 0]
gt_A[6, :] = [0, 0, 0, 1, 0, 0, 0]
gt_A

array([[0, 1, 1, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0, 0]])

In [42]:
result = """ 

"""

In [43]:
target_name = 'score'
benchmark_name = 'AG7'
exp_uid = '2024_02_01_1240_hk_Mixtral-8x7b-Groq'

# GPT-3.5-Turbo

https://poe.com/GPT-3.5-Turbo

https://poe.com/s/dOqXGdEGL7liQPf8GMCT

zeroshot https://poe.com/s/XuZyArxSChb4Ygv1lx9V

In [134]:
this_bot_name = "GPT-3.5-Turbo-v1933" 
anno_model  = "Mixtral-8x7b-Groq" 
V = set([target_name])
Z = set()
annotation[this_bot_name] = {}

## iteration 1

In [ ]:
print(propose_prompt_with_data_in_front(data))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data


## Group with 'Score' = -4

- This particular apple, while boasting a striking and consistent hue, falls short of expectations due to its unappealing small stature, an off-putting musty odor suggesting decay, and a flavor profile that tilts disappointingly toward sourness. The lack of juiciness further detracts from its desirability, making it a risky choice for suppliers and a potential loss-inducer in the market.
- Unfortunately, this batch of apples fails to meet quality standards. Their small size, extensive surface blemishes, and dry texture fall short of expectations. The off-putting musty scent suggests potential rot, and the sour taste overpowers any sweetness. Suppliers should beware; these apples could result in financial disappointment if offered to discerning customers.

## Group with 'Score' = -3

- Despite the abundant moisture offering a refreshing b

In [ ]:
result = """ 
<Factor Begin>

**Factor: Visual Appearance**
- 1: The apple has a vibrant, uniform color and minimal surface blemishes.
- 0: Otherwise; or not mentioned.
- -1: The apple has extensive surface blemishes, spots, and surface browning.

**Factor: Aroma**
- 1: The apple has a strong and inviting aroma.
- 0: Otherwise; or not mentioned.
- -1: The apple has a musty or rotten odor.

**Factor: Taste**
- 1: The apple leans towards sweetness and has a well-balanced flavor profile.
- 0: Otherwise; or not mentioned.
- -1: The apple leans towards sourness and lacks balanced sweetness.

**Factor: Texture**
- 1: The apple has a juicy and succulent texture.
- 0: Otherwise; or not mentioned.
- -1: The apple has a dry texture and lacks juiciness.

**Factor: Size**
- 1: The apple is within expectations in terms of size.
- 0: Otherwise; or not mentioned.
- -1: The apple is smaller than desired.

**Factor: Potential for Market Profitability**
- 1: The apple has the potential to meet profitability expectations in the market.
- 0: Otherwise; or not mentioned.
- -1: The apple is unlikely to meet profitability expectations in the market.

"""
factor_list = result.split('<Factor Begin>')[-1].split('\n\n')

factor_list = check_factor_list(factor_list)
factor_list

['visual appearance\n- 1: The apple has a vibrant, uniform color and minimal surface blemishes.\n- 0: Otherwise; or not mentioned.\n- -1: The apple has extensive surface blemishes, spots, and surface browning.',
 'aroma\n- 1: The apple has a strong and inviting aroma.\n- 0: Otherwise; or not mentioned.\n- -1: The apple has a musty or rotten odor.',
 'taste\n- 1: The apple leans towards sweetness and has a well-balanced flavor profile.\n- 0: Otherwise; or not mentioned.\n- -1: The apple leans towards sourness and lacks balanced sweetness.',
 'texture\n- 1: The apple has a juicy and succulent texture.\n- 0: Otherwise; or not mentioned.\n- -1: The apple has a dry texture and lacks juiciness.',
 'size\n- 1: The apple is within expectations in terms of size.\n- 0: Otherwise; or not mentioned.\n- -1: The apple is smaller than desired.',
 'potential for market profitability\n- 1: The apple has the potential to meet profitability expectations in the market.\n- 0: Otherwise; or not mentioned.\n

### annotation

In [ ]:
from utils import Logger
import time
anno_model  = "Mixtral-8x7b-Groq"  # "Llama-2-70b" #"Mistral-Medium"

sample_size = meta.shape[0]

for this_factor in factor_list:
    this_factor_name = get_factor_name(this_factor)

    if this_factor_name.lower() in '_'.join(annotation[this_bot_name].keys()).lower():
        continue
    
    print(this_factor_name)
    print(this_factor)

    logger = Logger(
        root = '/home/comp/cscxliu/COAT/AppleGastronome', 
        exp_name = f'Formal_exp_{exp_uid}_{benchmark_name}_anno_{this_factor_name}_for_{this_bot_name}_by_{anno_model}'
    )

    answers = []
    for idx in range(sample_size):
        print(f'\n ---- \n')
        anno_request = annotation_prompt(this_factor, meta.values[idx,-1].replace('\n', ' '))
        logger.straight_write('conversation', 'User: \n' + anno_request, mode='a')

        response = await chat(anno_request, bot_name=anno_model)
        logger.straight_write('conversation', f'{anno_model}: \n' + response, mode='a')

        this_value = get_value_from_responce(response)
        if this_value  in [None, '?']:
            this_value = 0

        answers.append(this_value)
        print(f'\n{idx+1}-th sample, factor "{this_factor_name}": {this_value}\n')
        time.sleep(0.2)
    annotation[this_bot_name][this_factor_name] = deepcopy(answers)

visual appearance
visual appearance
- 1: The apple has a vibrant, uniform color and minimal surface blemishes.
- 0: Otherwise; or not mentioned.
- -1: The apple has extensive surface blemishes, spots, and surface browning.

 ---- 


1-th sample, factor "visual appearance": 1


 ---- 


2-th sample, factor "visual appearance": 0


 ---- 


3-th sample, factor "visual appearance": 1


 ---- 


4-th sample, factor "visual appearance": 1


 ---- 


5-th sample, factor "visual appearance": 1


 ---- 


6-th sample, factor "visual appearance": 1


 ---- 


7-th sample, factor "visual appearance": 1


 ---- 


8-th sample, factor "visual appearance": -1


 ---- 


9-th sample, factor "visual appearance": 1


 ---- 


10-th sample, factor "visual appearance": 0


 ---- 


11-th sample, factor "visual appearance": 1


 ---- 


12-th sample, factor "visual appearance": -1


 ---- 


13-th sample, factor "visual appearance": -1


 ---- 


14-th sample, factor "visual appearance": -1


 ---- 


15

### confirmation

In [ ]:
iteration = 1
print(f'e_{exp_uid}_{this_bot_name}_iter{iteration}')
file_path = f"/home/comp/cscxliu/COAT/AppleGastronome/data/Apple_Gastronome_AG7_v20240201_data_interface_{exp_uid}_{this_bot_name}_iter{iteration}.xlsx"

e_2024_02_01_1240_hk_Mixtral-8x7b-Groq_GPT-3.5-Turbo-v1933_iter1


In [ ]:
# !!!
assert not os.path.exists(file_path)
annotation[this_bot_name][target_name] = meta[target_name].values
data_interface = pd.DataFrame(annotation[this_bot_name])
all_factors = list(data_interface.columns)

data_interface.to_excel(file_path, index=False) 

In [ ]:

data_interface = pd.read_excel(file_path)
all_factors = list(data_interface.columns)

for f in all_factors:
    annotation[this_bot_name][f] = deepcopy(data_interface[f].to_list())

In [ ]:
annotated_name = [target_name]+[f for idx, f in enumerate(all_factors) if (f != target_name)]
annoted_values = data_interface[annotated_name].values

ci_test = CIT(annoted_values)
alpha = 0.05

Sep = [idx for idx, f in enumerate(annotated_name) if (f in Z)and((f != target_name))]

valid_factors = [f for idx, f in enumerate(annotated_name) if (f not in V)and(ci_test(0,idx,Sep)<alpha)]
valid_factors

['size', 'aroma', 'flavor balance', 'juiciness']

In [ ]:
print(V==V.union(valid_factors))
V=V.union(valid_factors)
V

False


{'aroma', 'flavor balance', 'juiciness', 'score', 'size'}

In [ ]:
factors = [ f for f in V if f!= target_name]
annotated_name = [target_name]+factors
annoted_values = data_interface[annotated_name].values

# Causal Discovery (FCI)
g, edges = fci(annoted_values, alpha = 0.05, verbose=False)

# visualization
pdy = GraphUtils.to_pydot(g, labels=annotated_name)
print(annoted_values.std(0))
print(pdy.to_string())

Depth=0, working on node 5: 100%|██████████| 6/6 [00:00<00:00, 906.78it/s]

X1 --> X5
[1.82756669 0.9343982  0.89442719 0.89994444 0.90614568 0.92390476]
digraph  {
dpi=200;
fontsize=18;
0 [label=X1];
0 [label=score];
1 [label=X2];
1 [label=taste];
2 [label=X3];
2 [label=texture];
3 [label=X4];
3 [label=aroma];
4 [label=X5];
4 [label="potential for market profitability"];
5 [label=X6];
5 [label=size];
1 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
3 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
0 -> 4  [arrowhead=normal, arrowtail=none, dir=both];
5 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
1 -> 2  [arrowhead=odot, arrowtail=odot, dir=both];
3 -> 4  [arrowhead=normal, arrowtail=none, dir=both];
}



In [ ]:
Z = GetMB(g.graph, annotated_name)
Z

array(['taste', 'aroma', 'potential for market profitability', 'size'],
      dtype='<U34')

## iteration 2

### feedback

In [ ]:
focus_factor = 'score'
focus_label = data_interface[focus_factor].values[:100]
background_factor = [f for f in list(Z) if f not in [focus_factor]]
cond_vectors = data_interface[background_factor].values[:100,:]

g = KMeans(n_clusters=min(len(factors), 4), random_state=0, n_init="auto").fit(cond_vectors).labels_

entropy_values = []
set_g = list(set(g))
for g_idx in set_g:
    qk = np.unique(focus_label[g==g_idx], return_counts=True)[1]
    n = qk.sum()
    qk = qk/ n
    this_entropy_values = stats.entropy(qk)
    if n < len(qk) * 3:
        this_entropy_values = -1
    entropy_values.append(this_entropy_values)
    print(g_idx, entropy_values[-1],n, qk)

g_chosen = set_g[np.argmax(entropy_values)]
print(f'choose g=={g_chosen}')


0 1.0509134745090587 25 [0.2  0.36 0.44]
1 1.5783439262040448 39 [0.05128205 0.20512821 0.25641026 0.33333333 0.1025641  0.05128205]
2 1.5684705556118042 25 [0.16 0.2  0.24 0.28 0.12]
3 -1 11 [0.09090909 0.09090909 0.09090909 0.54545455 0.18181818]
choose g==1


In [ ]:
example = ''

chosen_idx = g==g_chosen
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,-1]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example, [f  for f in data_interface.columns if f not in [target_name]]))



You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'score' = 0

---

 This compact apple surprises with a delightful sweetness that overshadows its tartness, offering a burst of refreshing moisture with each bite. However, its diminutive size could pose a risk to suppliers and buyers alike, as it may not meet profit expectations, potentially causing financial disappointment.

---

 This apple, with its lively and consistent hue, delights visually, while its petit frame suggests a charming snack size. The taste profile leans towards sweet, coupled with ample juiciness for a hydrating bite, offering a pleasurable eating experience. However, economic caution is advised, as these apples might disappoint commercially, potentially not meeting financial projections and risking monetary setbacks.

---

 Despite its vibrant, uniform coloring and large size, this apple's quality is undermined by a significant 

In [ ]:
result = """ 
<Factor Begin>

**Freshness**
- 1: The apple has a fresh and appealing aroma and taste, indicating good quality and high freshness.
- 0: Otherwise; or not mentioned.
- -1: The apple has a musty or rotten scent, suggesting poor freshness and compromised quality.

**Aesthetics**
- 1: The apple has a vibrant and uniform color, visually appealing and indicating good quality.
- 0: Otherwise; or not mentioned.
- -1: The apple has spots, browning, or surface blemishes, detracting from its visual appeal and suggesting poor quality.

**Size**
- 1: The apple is large in size, which may be perceived as desirable by some consumers.
- 0: Otherwise; or not mentioned.
- -1: The apple is small in size, which may be perceived as undesirable by some consumers.

**Sweetness**
- 1: The apple has a sweet flavor profile, balancing with other taste elements.
- 0: Otherwise; or not mentioned.
- -1: The apple has a sour flavor profile, dominating over sweetness and potentially disappointing consumers.




"""
factor_list = result.split('<Factor Begin>')[-1].split('\n\n')

factor_list = check_factor_list(factor_list)
factor_list

['freshness\n- 1: The apple has a fresh and appealing aroma and taste, indicating good quality and high freshness.\n- 0: Otherwise; or not mentioned.\n- -1: The apple has a musty or rotten scent, suggesting poor freshness and compromised quality.',
 'aesthetics\n- 1: The apple has a vibrant and uniform color, visually appealing and indicating good quality.\n- 0: Otherwise; or not mentioned.\n- -1: The apple has spots, browning, or surface blemishes, detracting from its visual appeal and suggesting poor quality.',
 'size\n- 1: The apple is large in size, which may be perceived as desirable by some consumers.\n- 0: Otherwise; or not mentioned.\n- -1: The apple is small in size, which may be perceived as undesirable by some consumers.',
 'sweetness\n- 1: The apple has a sweet flavor profile, balancing with other taste elements.\n- 0: Otherwise; or not mentioned.\n- -1: The apple has a sour flavor profile, dominating over sweetness and potentially disappointing consumers.']

### annotation

In [ ]:
from utils import Logger
import time
anno_model  = "Mixtral-8x7b-Groq" # "Llama-2-70b"  # "Llama-2-70b" #"Mistral-Medium"

sample_size = meta.shape[0]

for this_factor in factor_list:
    this_factor_name = get_factor_name(this_factor)
    
    if this_factor_name.lower() in '_'.join(annotation[this_bot_name].keys()).lower():
        print(f'skip {this_factor_name}')
        continue
    print(this_factor_name)
    print(this_factor)

    logger = Logger(
        root = '/home/comp/cscxliu/COAT/AppleGastronome', 
        exp_name = f'Formal_exp_{exp_uid}_{benchmark_name}_anno_{this_factor_name}_for_{this_bot_name}_by_{anno_model}'
    )

    answers = []
    for idx in range(sample_size):
        print(f'\n ---- \n')
        anno_request = annotation_prompt(this_factor, meta.values[idx,-1].replace('\n', ' '))
        logger.straight_write('conversation', 'User: \n' + anno_request, mode='a')

        response = await chat(anno_request, bot_name=anno_model)
        logger.straight_write('conversation', f'{anno_model}: \n' + response, mode='a')

        this_value = get_value_from_responce(response)
        if this_value  in [None, '?']:
            this_value = 0

        answers.append(this_value)
        print(f'\n{idx+1}-th sample, factor "{this_factor_name}": {this_value}\n')
        time.sleep(0.2)
    annotation[this_bot_name][this_factor_name] = deepcopy(answers)

freshness
freshness
- 1: The apple has a fresh and appealing aroma and taste, indicating good quality and high freshness.
- 0: Otherwise; or not mentioned.
- -1: The apple has a musty or rotten scent, suggesting poor freshness and compromised quality.

 ---- 




1-th sample, factor "freshness": -1


 ---- 


2-th sample, factor "freshness": -1


 ---- 


3-th sample, factor "freshness": 1


 ---- 


4-th sample, factor "freshness": 0


 ---- 


5-th sample, factor "freshness": -1


 ---- 


6-th sample, factor "freshness": -1


 ---- 


7-th sample, factor "freshness": 1


 ---- 


8-th sample, factor "freshness": 1


 ---- 


9-th sample, factor "freshness": 0


 ---- 


10-th sample, factor "freshness": -1


 ---- 


11-th sample, factor "freshness": -1


 ---- 


12-th sample, factor "freshness": -1


 ---- 


13-th sample, factor "freshness": 0


 ---- 


14-th sample, factor "freshness": 0


 ---- 


15-th sample, factor "freshness": -1


 ---- 


16-th sample, factor "freshness": 1


 ---- 


17-th sample, factor "freshness": -1


 ---- 


18-th sample, factor "freshness": 0


 ---- 


19-th sample, factor "freshness": -1


 ---- 


20-th sample, factor "freshness": -1


 ---- 


21-th sample, factor "freshness": 1


 ---- 


22-th samp

### confirmation

In [ ]:
iteration = 2
print(exp_uid, this_bot_name, iteration)
file_path = f"/home/comp/cscxliu/COAT/AppleGastronome/data/Apple_Gastronome_AG7_v20240201_data_interface_{exp_uid}_{this_bot_name}_iter{iteration}.xlsx"


2024_02_01_1240_hk_Mixtral-8x7b-Groq GPT-3.5-Turbo-v1933 2


In [ ]:
# !!!
assert not os.path.exists(file_path)
annotation[this_bot_name][target_name] = meta[target_name].values
data_interface = pd.DataFrame(annotation[this_bot_name])
all_factors = list(data_interface.columns)

data_interface.to_excel(file_path, index=False) 

In [ ]:

data_interface = pd.read_excel(file_path)
all_factors = list(data_interface.columns)

for f in all_factors:
    annotation[this_bot_name][f] = deepcopy(data_interface[f].to_list())

In [ ]:
annotated_name = [target_name]+[f for idx, f in enumerate(all_factors) if (f != target_name)]
annoted_values = data_interface[annotated_name].values

ci_test = CIT(annoted_values)
alpha = 0.05

Sep = [idx for idx, f in enumerate(annotated_name) if (f in Z)and((f != target_name))]

valid_factors = [f for idx, f in enumerate(annotated_name) if (f not in V)and(ci_test(0,idx,Sep)<alpha)]
valid_factors

['aesthetics']

In [ ]:
print(V==V.union(valid_factors))
V=V.union(valid_factors)
V

False


{'aesthetics',
 'aroma',
 'potential for market profitability',
 'score',
 'size',
 'taste',
 'texture'}

In [ ]:
factors = [ f for f in V if f!= target_name]
annotated_name = [target_name]+factors
annoted_values = data_interface[annotated_name].values

# Causal Discovery (FCI)
g, edges = fci(annoted_values, alpha = 0.05, verbose=False)

# visualization
pdy = GraphUtils.to_pydot(g, labels=annotated_name)
print(annoted_values.std(0))
print(pdy.to_string())

Depth=0, working on node 6: 100%|██████████| 7/7 [00:00<00:00, 847.87it/s]

X1 --> X4
[1.82756669 0.9343982  0.89442719 0.90614568 0.89994444 0.92390476
 0.96389833]
digraph  {
dpi=200;
fontsize=18;
0 [label=X1];
0 [label=score];
1 [label=X2];
1 [label=taste];
2 [label=X3];
2 [label=texture];
3 [label=X4];
3 [label="potential for market profitability"];
4 [label=X5];
4 [label=aroma];
5 [label=X6];
5 [label=size];
6 [label=X7];
6 [label=aesthetics];
1 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
0 -> 3  [arrowhead=normal, arrowtail=none, dir=both];
4 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
5 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
1 -> 2  [arrowhead=odot, arrowtail=odot, dir=both];
4 -> 3  [arrowhead=normal, arrowtail=none, dir=both];
6 -> 3  [arrowhead=normal, arrowtail=odot, dir=both];
}



In [ ]:
Z = GetMB(g.graph, annotated_name)
Z

array(['taste', 'potential for market profitability', 'aroma', 'size',
       'aesthetics'], dtype='<U34')

## iteration 3

In [ ]:
focus_factor = 'score'
focus_label = data_interface[focus_factor].values[:100]
background_factor = [f for f in list(Z) if f not in [focus_factor]]
cond_vectors = data_interface[background_factor].values[:100,:]

g = KMeans(n_clusters=min(len(factors), 4), random_state=0, n_init="auto").fit(cond_vectors).labels_

entropy_values = []
set_g = list(set(g))
for g_idx in set_g:
    qk = np.unique(focus_label[g==g_idx], return_counts=True)[1]
    n = qk.sum()
    qk = qk/ n
    this_entropy_values = stats.entropy(qk)
    if n < len(qk) * 3:
        this_entropy_values = -1
    entropy_values.append(this_entropy_values)
    print(g_idx, entropy_values[-1],n, qk)

g_chosen = set_g[np.argmax(entropy_values)]
print(f'choose g=={g_chosen}')


0 0.8823109245150755 15 [0.4        0.53333333 0.06666667]
1 1.343819601921041 24 [0.04166667 0.33333333 0.25       0.33333333 0.04166667]
2 1.663457138315657 40 [0.05  0.2   0.25  0.275 0.125 0.1  ]
3 1.4629537629977367 21 [0.04761905 0.19047619 0.28571429 0.33333333 0.14285714]
choose g==2


In [ ]:
example = ''

chosen_idx = g==g_chosen
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,-1]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example, [f  for f in data_interface.columns if f not in [target_name]]))



You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'score' = 0

---

 Despite its vibrant, uniform coloring and large size, this apple's quality is undermined by a significant flaw: a musty or rotten taste, skewing more sour than sweet. Its dryness suggests a lack of essential moisture, which not only impairs the eating experience but also lessens its commercial appeal, potentially resulting in disappointing sales and unforeseen losses for suppliers and customers alike.

---

 This apple impresses with its vibrant, uniform color and generous size, potentially appealing to eye-catching displays. However, the taste leans towards sour, which may not satisfy a sweet-seeking palate. Its ample juice offers a refreshing bite, yet the flavor profile suggests it could disappoint profit expectations, posing a financial risk to suppliers and consumers alike.

---

 The apple in question disappoints with a pleth

In [ ]:
result = """ 
<Factor Begin>

**Factor 1: External Appearance**
- 1: The apple has a vibrant and uniform color.
- 0: Otherwise; or not mentioned.
- -1: The apple has numerous surface blemishes and small stature, compromising its visual appeal.

**Factor 2: Internal Quality**
- 1: The apple has a robust aroma hinting at freshness.
- 0: Otherwise; or not mentioned.
- -1: The apple has a musty or rotten scent, indicating potential spoilage.

**Factor 3: Flavor Profile**
- 1: The apple has a pleasant, balanced sweetness.
- 0: Otherwise; or not mentioned.
- -1: The apple has an overpowering sourness, detracting from its taste.

"""
factor_list = result.split('<Factor Begin>')[-1].split('\n\n')

factor_list = check_factor_list(factor_list)
factor_list

['external appearance\n- 1: The apple has a vibrant and uniform color.\n- 0: Otherwise; or not mentioned.\n- -1: The apple has numerous surface blemishes and small stature, compromising its visual appeal.',
 'internal quality\n- 1: The apple has a robust aroma hinting at freshness.\n- 0: Otherwise; or not mentioned.\n- -1: The apple has a musty or rotten scent, indicating potential spoilage.',
 'flavor profile\n- 1: The apple has a pleasant, balanced sweetness.\n- 0: Otherwise; or not mentioned.\n- -1: The apple has an overpowering sourness, detracting from its taste.']

### annotation

In [ ]:
from utils import Logger
import time
anno_model  = "Mixtral-8x7b-Groq" # "Llama-2-70b"  # "Llama-2-70b" #"Mistral-Medium"

sample_size = meta.shape[0]

for this_factor in factor_list:
    this_factor_name = get_factor_name(this_factor)
    
    if this_factor_name.lower() in '_'.join(annotation[this_bot_name].keys()).lower():
        print(f'skip {this_factor_name}')
        continue
    print(this_factor_name)
    print(this_factor)

    logger = Logger(
        root = '/home/comp/cscxliu/COAT/AppleGastronome', 
        exp_name = f'Formal_exp_{exp_uid}_{benchmark_name}_anno_{this_factor_name}_for_{this_bot_name}_by_{anno_model}'
    )

    answers = []
    for idx in range(sample_size):
        print(f'\n ---- \n')
        anno_request = annotation_prompt(this_factor, meta.values[idx,-1].replace('\n', ' '))
        logger.straight_write('conversation', 'User: \n' + anno_request, mode='a')

        response = await chat(anno_request, bot_name=anno_model)
        logger.straight_write('conversation', f'{anno_model}: \n' + response, mode='a')

        this_value = get_value_from_responce(response)
        if this_value  in [None, '?']:
            this_value = 0

        answers.append(this_value)
        print(f'\n{idx+1}-th sample, factor "{this_factor_name}": {this_value}\n')
        time.sleep(0.2)
    annotation[this_bot_name][this_factor_name] = deepcopy(answers)

external appearance
external appearance
- 1: The apple has a vibrant and uniform color.
- 0: Otherwise; or not mentioned.
- -1: The apple has numerous surface blemishes and small stature, compromising its visual appeal.

 ---- 




1-th sample, factor "external appearance": 1


 ---- 


2-th sample, factor "external appearance": 0


 ---- 


3-th sample, factor "external appearance": 1


 ---- 


4-th sample, factor "external appearance": 1


 ---- 


5-th sample, factor "external appearance": -1


 ---- 


6-th sample, factor "external appearance": 1


 ---- 


7-th sample, factor "external appearance": 1


 ---- 


8-th sample, factor "external appearance": -1


 ---- 


9-th sample, factor "external appearance": 1


 ---- 


10-th sample, factor "external appearance": -1


 ---- 


11-th sample, factor "external appearance": 1


 ---- 


12-th sample, factor "external appearance": -1


 ---- 


13-th sample, factor "external appearance": -1


 ---- 


14-th sample, factor "external appearance": -1


 ---- 


15-th sample, factor "external appearance": -1


 ---- 


16-th sample, factor "external appearance": 1


 ---- 


17-th sample, factor "external appearance": -1


 ---- 


18-th sample, factor "external 

### confirmation

In [ ]:
iteration = 3
print(exp_uid, this_bot_name, iteration)
file_path = f"/home/comp/cscxliu/COAT/AppleGastronome/data/Apple_Gastronome_AG7_v20240201_data_interface_{exp_uid}_{this_bot_name}_iter{iteration}.xlsx"


2024_02_01_1240_hk_Mixtral-8x7b-Groq GPT-3.5-Turbo-v1933 3


In [ ]:
# !!!
assert not os.path.exists(file_path)
annotation[this_bot_name][target_name] = meta[target_name].values
data_interface = pd.DataFrame(annotation[this_bot_name])
all_factors = list(data_interface.columns)

data_interface.to_excel(file_path, index=False) 

In [ ]:

data_interface = pd.read_excel(file_path)
all_factors = list(data_interface.columns)

for f in all_factors:
    annotation[this_bot_name][f] = deepcopy(data_interface[f].to_list())

In [ ]:
annotated_name = [target_name]+[f for idx, f in enumerate(all_factors) if (f != target_name)]
annoted_values = data_interface[annotated_name].values

ci_test = CIT(annoted_values)
alpha = 0.05

Sep = [idx for idx, f in enumerate(annotated_name) if (f in Z)and((f != target_name))]

valid_factors = [f for idx, f in enumerate(annotated_name) if (f not in V)and(ci_test(0,idx,Sep)<alpha)]
valid_factors

['internal quality', 'flavor profile']

In [ ]:
print(V==V.union(valid_factors))
V=V.union(valid_factors)
V

False


{'aesthetics',
 'aroma',
 'flavor profile',
 'internal quality',
 'potential for market profitability',
 'score',
 'size',
 'taste',
 'texture'}

In [ ]:
factors = [ f for f in V if f!= target_name]
annotated_name = [target_name]+factors
annoted_values = data_interface[annotated_name].values

# Causal Discovery (FCI)
g, edges = fci(annoted_values, alpha = 0.05, verbose=False)

# visualization
pdy = GraphUtils.to_pydot(g, labels=annotated_name)
print(annoted_values.std(0))
print(pdy.to_string())

Depth=0, working on node 8: 100%|██████████| 9/9 [00:00<00:00, 719.42it/s]

X1 --> X4
[1.82756669 0.93037627 0.9343982  0.90614568 0.88741197 0.89442719
 0.89994444 0.96389833 0.92390476]
digraph  {
dpi=200;
fontsize=18;
0 [label=X1];
0 [label=score];
1 [label=X2];
1 [label="flavor profile"];
2 [label=X3];
2 [label=taste];
3 [label=X4];
3 [label="potential for market profitability"];
4 [label=X5];
4 [label="internal quality"];
5 [label=X6];
5 [label=texture];
6 [label=X7];
6 [label=aroma];
7 [label=X8];
7 [label=aesthetics];
8 [label=X9];
8 [label=size];
2 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
0 -> 3  [arrowhead=normal, arrowtail=none, dir=both];
8 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
1 -> 2  [arrowhead=odot, arrowtail=odot, dir=both];
1 -> 5  [arrowhead=odot, arrowtail=odot, dir=both];
6 -> 3  [arrowhead=normal, arrowtail=odot, dir=both];
7 -> 3  [arrowhead=normal, arrowtail=odot, dir=both];
4 -> 6  [arrowhead=odot, arrowtail=odot, dir=both];
}



In [ ]:
Z = GetMB(g.graph, annotated_name)
Z

array(['taste', 'potential for market profitability', 'aroma',
       'aesthetics', 'size'], dtype='<U34')

## iteration 4

In [ ]:
focus_factor = 'score'
focus_label = data_interface[focus_factor].values[:100]
background_factor = [f for f in list(Z) if f not in [focus_factor]]
cond_vectors = data_interface[background_factor].values[:100,:]

g = KMeans(n_clusters=min(len(factors), 4), random_state=0, n_init="auto").fit(cond_vectors).labels_

entropy_values = []
set_g = list(set(g))
for g_idx in set_g:
    qk = np.unique(focus_label[g==g_idx], return_counts=True)[1]
    n = qk.sum()
    qk = qk/ n
    this_entropy_values = stats.entropy(qk)
    if n < len(qk) * 3:
        this_entropy_values = -1
    entropy_values.append(this_entropy_values)
    print(g_idx, entropy_values[-1],n, qk)

g_chosen = set_g[np.argmax(entropy_values)]
print(f'choose g=={g_chosen}')


0 1.669479015739323 22 [0.04545455 0.18181818 0.27272727 0.22727273 0.18181818 0.09090909]
1 1.3499473197675713 22 [0.04545455 0.31818182 0.22727273 0.36363636 0.04545455]
2 1.686300462793552 24 [0.04166667 0.16666667 0.16666667 0.29166667 0.16666667 0.16666667]
3 1.5403727951469244 32 [0.15625 0.3125  0.21875 0.21875 0.09375]
choose g==2


In [ ]:
example = ''

chosen_idx = g==g_chosen
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,-1]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example, [f  for f in data_interface.columns if f not in [target_name]]))



You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'score' = 0

---

 This apple, despite its appealing large size and abundant juiciness, fails to meet the mark due to significant surface blemishing and a predominantly sour flavor profile, diverging from the usual sweetness preferred by consumers. The visible spots and tendency towards tartness hint at a constrained market appeal, potentially resulting in disappointing sales outcomes for suppliers.

---

 Despite its inviting strong aroma, this apple's numerous surface blemishes and small stature disappoint. The dryness and lack of moisture further detract from the fruit's appeal, signaling a subpar eating experience. Given these shortcomings, it's unlikely these apples will meet profitability targets, potentially leading to financial losses for suppliers and unsatisfactory purchases for customers.

---

 The apple in question disappoints with a ple

In [ ]:
result = """ 
<Factor Begin>

**Factor Name: Aesthetic Appeal**
- 1: The apple has an appealing visual appearance with minimal surface blemishes, spots, and browning.
- 0: Otherwise; or not mentioned.
- -1: The apple has significant surface blemishing, spots, and browning, indicating poor aesthetics and potential quality issues.

**Factor Name: Flavor Profile**
- 1: The apple has a balanced flavor profile, with a desirable level of sweetness and sourness.
- 0: Otherwise; or not mentioned.
- -1: The apple has an unbalanced flavor profile, leaning towards sourness and lacking the anticipated sweetness.

**Factor Name: Texture**
- 1: The apple has a desirable texture, with good juiciness and a satisfying mouthfeel.
- 0: Otherwise; or not mentioned.
- -1: The apple has an undesirable texture, being overly dry and lacking juiciness.

**Factor Name: Market Performance**
- 1: The apple has a high potential for market profitability, meeting commercial expectations and generating satisfactory returns for suppliers.
- 0: Otherwise; or not mentioned.
- -1: The apple has a low potential for market profitability, posing a risk of financial loss for suppliers and unsatisfactory purchases for customers.

"""
factor_list = result.split('<Factor Begin>')[-1].split('\n\n')

factor_list = check_factor_list(factor_list)
factor_list

['aesthetic appeal\n- 1: The apple has an appealing visual appearance with minimal surface blemishes, spots, and browning.\n- 0: Otherwise; or not mentioned.\n- -1: The apple has significant surface blemishing, spots, and browning, indicating poor aesthetics and potential quality issues.',
 'flavor profile\n- 1: The apple has a balanced flavor profile, with a desirable level of sweetness and sourness.\n- 0: Otherwise; or not mentioned.\n- -1: The apple has an unbalanced flavor profile, leaning towards sourness and lacking the anticipated sweetness.',
 'texture\n- 1: The apple has a desirable texture, with good juiciness and a satisfying mouthfeel.\n- 0: Otherwise; or not mentioned.\n- -1: The apple has an undesirable texture, being overly dry and lacking juiciness.',
 'market performance\n- 1: The apple has a high potential for market profitability, meeting commercial expectations and generating satisfactory returns for suppliers.\n- 0: Otherwise; or not mentioned.\n- -1: The apple has

### annotation

In [ ]:
from utils import Logger
import time
anno_model  = "Mixtral-8x7b-Groq" # "Llama-2-70b"  # "Llama-2-70b" #"Mistral-Medium"

sample_size = meta.shape[0]

for this_factor in factor_list:
    this_factor_name = get_factor_name(this_factor)
    
    if this_factor_name.lower() in '_'.join(annotation[this_bot_name].keys()).lower():
        print(f'skip {this_factor_name}')
        continue
    print(this_factor_name)
    print(this_factor)

    logger = Logger(
        root = '/home/comp/cscxliu/COAT/AppleGastronome', 
        exp_name = f'Formal_exp_{exp_uid}_{benchmark_name}_anno_{this_factor_name}_for_{this_bot_name}_by_{anno_model}'
    )

    answers = []
    for idx in range(sample_size):
        print(f'\n ---- \n')
        anno_request = annotation_prompt(this_factor, meta.values[idx,-1].replace('\n', ' '))
        logger.straight_write('conversation', 'User: \n' + anno_request, mode='a')

        response = await chat(anno_request, bot_name=anno_model)
        logger.straight_write('conversation', f'{anno_model}: \n' + response, mode='a')

        this_value = get_value_from_responce(response)
        if this_value  in [None, '?']:
            this_value = 0

        answers.append(this_value)
        print(f'\n{idx+1}-th sample, factor "{this_factor_name}": {this_value}\n')
        time.sleep(0.2)
    annotation[this_bot_name][this_factor_name] = deepcopy(answers)

aesthetic appeal
aesthetic appeal
- 1: The apple has an appealing visual appearance with minimal surface blemishes, spots, and browning.
- 0: Otherwise; or not mentioned.
- -1: The apple has significant surface blemishing, spots, and browning, indicating poor aesthetics and potential quality issues.

 ---- 




1-th sample, factor "aesthetic appeal": 1


 ---- 


2-th sample, factor "aesthetic appeal": 1


 ---- 


3-th sample, factor "aesthetic appeal": -1


 ---- 


4-th sample, factor "aesthetic appeal": 0


 ---- 


5-th sample, factor "aesthetic appeal": 1


 ---- 


6-th sample, factor "aesthetic appeal": 1


 ---- 


7-th sample, factor "aesthetic appeal": 1


 ---- 


8-th sample, factor "aesthetic appeal": 0


 ---- 


9-th sample, factor "aesthetic appeal": 1


 ---- 


10-th sample, factor "aesthetic appeal": -1


 ---- 


11-th sample, factor "aesthetic appeal": 0


 ---- 


12-th sample, factor "aesthetic appeal": -1


 ---- 


13-th sample, factor "aesthetic appeal": -1


 ---- 


14-th sample, factor "aesthetic appeal": -1


 ---- 


15-th sample, factor "aesthetic appeal": -1


 ---- 


16-th sample, factor "aesthetic appeal": 1


 ---- 


17-th sample, factor "aesthetic appeal": -1


 ---- 


18-th sample, factor "aesthetic appeal": 1


 ---- 


19-th sample, factor "aesthet

### confirmation

In [ ]:
iteration = 4
print(exp_uid, this_bot_name, iteration)
file_path = f"/home/comp/cscxliu/COAT/AppleGastronome/data/Apple_Gastronome_AG7_v20240201_data_interface_{exp_uid}_{this_bot_name}_iter{iteration}.xlsx"


2024_02_01_1240_hk_Mixtral-8x7b-Groq GPT-3.5-Turbo-v1933 4


In [ ]:
# !!!
assert not os.path.exists(file_path)
annotation[this_bot_name][target_name] = meta[target_name].values
data_interface = pd.DataFrame(annotation[this_bot_name])
all_factors = list(data_interface.columns)

data_interface.to_excel(file_path, index=False) 

In [ ]:

data_interface = pd.read_excel(file_path)
all_factors = list(data_interface.columns)

for f in all_factors:
    annotation[this_bot_name][f] = deepcopy(data_interface[f].to_list())

In [ ]:
annotated_name = [target_name]+[f for idx, f in enumerate(all_factors) if (f != target_name)]
annoted_values = data_interface[annotated_name].values

ci_test = CIT(annoted_values)
alpha = 0.05

Sep = [idx for idx, f in enumerate(annotated_name) if (f in Z)and((f != target_name))]

valid_factors = [f for idx, f in enumerate(annotated_name) if (f not in V)and(ci_test(0,idx,Sep)<alpha)]
valid_factors

[]

In [ ]:
print(V==V.union(valid_factors))
V=V.union(valid_factors)
V

True


{'aesthetics',
 'aroma',
 'flavor profile',
 'internal quality',
 'potential for market profitability',
 'score',
 'size',
 'taste',
 'texture'}

In [ ]:
factors = [ f for f in V if f!= target_name]
annotated_name = [target_name]+factors
annoted_values = data_interface[annotated_name].values

# Causal Discovery (FCI)
g, edges = fci(annoted_values, alpha = 0.05, verbose=False)

# visualization
pdy = GraphUtils.to_pydot(g, labels=annotated_name)
print(annoted_values.std(0))
print(pdy.to_string())

Depth=0, working on node 8: 100%|██████████| 9/9 [00:00<00:00, 737.44it/s]

X1 --> X7
[1.82756669 0.93037627 0.9343982  0.89442719 0.89994444 0.96389833
 0.90614568 0.92390476 0.88741197]
digraph  {
dpi=200;
fontsize=18;
0 [label=X1];
0 [label=score];
1 [label=X2];
1 [label="flavor profile"];
2 [label=X3];
2 [label=taste];
3 [label=X4];
3 [label=texture];
4 [label=X5];
4 [label=aroma];
5 [label=X6];
5 [label=aesthetics];
6 [label=X7];
6 [label="potential for market profitability"];
7 [label=X8];
7 [label=size];
8 [label=X9];
8 [label="internal quality"];
2 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
0 -> 6  [arrowhead=normal, arrowtail=none, dir=both];
7 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
1 -> 2  [arrowhead=odot, arrowtail=odot, dir=both];
1 -> 3  [arrowhead=odot, arrowtail=odot, dir=both];
4 -> 6  [arrowhead=normal, arrowtail=odot, dir=both];
4 -> 8  [arrowhead=odot, arrowtail=odot, dir=both];
5 -> 6  [arrowhead=normal, arrowtail=odot, dir=both];
}



In [ ]:
Z = GetMB(g.graph, annotated_name)
Z

array(['taste', 'aroma', 'aesthetics',
       'potential for market profitability', 'size'], dtype='<U34')

# GPT-4-Turbo

https://poe.com/GPT-4

https://poe.com/s/aQ0rRnoBOsIrtU2GRljR

zero-shot https://poe.com/s/lFeRoWVNO7TgtP6CoC4w

In [135]:
this_bot_name = "GPT-3.5-Turbo-v1933" 
anno_model  = "Mixtral-8x7b-Groq" 
V = set([target_name])
Z = set()
annotation[this_bot_name] = {}

## iteration 1

In [20]:
print(propose_prompt_with_data_in_front(data))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data


## Group with 'Score' = -4

- This particular apple, while boasting a striking and consistent hue, falls short of expectations due to its unappealing small stature, an off-putting musty odor suggesting decay, and a flavor profile that tilts disappointingly toward sourness. The lack of juiciness further detracts from its desirability, making it a risky choice for suppliers and a potential loss-inducer in the market.
- Unfortunately, this batch of apples fails to meet quality standards. Their small size, extensive surface blemishes, and dry texture fall short of expectations. The off-putting musty scent suggests potential rot, and the sour taste overpowers any sweetness. Suppliers should beware; these apples could result in financial disappointment if offered to discerning customers.

## Group with 'Score' = -3

- Sadly, this apple batch underwhelms, presenting numer

In [21]:
result = """ 
<Factor Begin>

**Size**
- 1: Described as "large" or "substantial size".
- 0: Otherwise; or not mentioned.
- -1: Described as "small stature" or "small size".

**Aroma**
- 1: Described as "strong", "inviting", or "potent" fragrance/aroma.
- 0: Otherwise; or not mentioned.
- -1: Described as "musty", "off-putting", or "rotten" odor/scent.

**Flavor Balance**
- 1: The sweetness is emphasized over sourness, or sourness is mentioned as a hint/minor compared to sweetness.
- 0: Otherwise; or not mentioned.
- -1: The sourness is emphasized over sweetness, or sweetness is mentioned as inadequate compared to sourness.

**Juiciness**
- 1: Described as "abundant moisture", "refreshing bite", or "juicy".
- 0: Otherwise; or not mentioned.
- -1: Described as "dry texture", "lack of juiciness", or "dryness".

**Appearance**
- 1: Described as "vibrant, uniform color" without negative mentions of blemishes.
- 0: Otherwise; or not mentioned.
- -1: Described as having "surface blemishes", "spots", "browning", or lacking uniformity.

"""
factor_list = result.split('<Factor Begin>')[-1].split('\n\n')

factor_list = check_factor_list(factor_list)
factor_list

['size\n- 1: Described as "large" or "substantial size".\n- 0: Otherwise; or not mentioned.\n- -1: Described as "small stature" or "small size".',
 'aroma\n- 1: Described as "strong", "inviting", or "potent" fragrance/aroma.\n- 0: Otherwise; or not mentioned.\n- -1: Described as "musty", "off-putting", or "rotten" odor/scent.',
 'flavor balance\n- 1: The sweetness is emphasized over sourness, or sourness is mentioned as a hint/minor compared to sweetness.\n- 0: Otherwise; or not mentioned.\n- -1: The sourness is emphasized over sweetness, or sweetness is mentioned as inadequate compared to sourness.',
 'juiciness\n- 1: Described as "abundant moisture", "refreshing bite", or "juicy".\n- 0: Otherwise; or not mentioned.\n- -1: Described as "dry texture", "lack of juiciness", or "dryness".',
 'appearance\n- 1: Described as "vibrant, uniform color" without negative mentions of blemishes.\n- 0: Otherwise; or not mentioned.\n- -1: Described as having "surface blemishes", "spots", "browning", 

### annotation

In [22]:
from utils import Logger
import time
anno_model  = "Mixtral-8x7b-Groq"  # "Llama-2-70b" #"Mistral-Medium"

sample_size = meta.shape[0]

for this_factor in factor_list:
    this_factor_name = get_factor_name(this_factor)

    if this_factor_name.lower() in '_'.join(annotation[this_bot_name].keys()).lower():
        continue
    
    print(this_factor_name)
    print(this_factor)

    logger = Logger(
        root = '/home/comp/cscxliu/COAT/AppleGastronome', 
        exp_name = f'Formal_exp_{exp_uid}_{benchmark_name}_anno_{this_factor_name}_for_{this_bot_name}_by_{anno_model}'
    )

    answers = []
    for idx in range(sample_size):
        print(f'\n ---- \n')
        anno_request = annotation_prompt(this_factor, meta.values[idx,-1].replace('\n', ' '))
        logger.straight_write('conversation', 'User: \n' + anno_request, mode='a')

        response = await chat(anno_request, bot_name=anno_model)
        logger.straight_write('conversation', f'{anno_model}: \n' + response, mode='a')

        this_value = get_value_from_responce(response)
        if this_value  in [None, '?']:
            this_value = 0

        answers.append(this_value)
        print(f'\n{idx+1}-th sample, factor "{this_factor_name}": {this_value}\n')
        time.sleep(0.2)
    annotation[this_bot_name][this_factor_name] = deepcopy(answers)

size
size
- 1: Described as "large" or "substantial size".
- 0: Otherwise; or not mentioned.
- -1: Described as "small stature" or "small size".

 ---- 


1-th sample, factor "size": -1


 ---- 


2-th sample, factor "size": -1


 ---- 


3-th sample, factor "size": 1


 ---- 


4-th sample, factor "size": 0


 ---- 


5-th sample, factor "size": -1


 ---- 


6-th sample, factor "size": 0


 ---- 


7-th sample, factor "size": 1


 ---- 


8-th sample, factor "size": -1


 ---- 


9-th sample, factor "size": 0


 ---- 


10-th sample, factor "size": -1


 ---- 


11-th sample, factor "size": 1


 ---- 


12-th sample, factor "size": -1


 ---- 


13-th sample, factor "size": -1


 ---- 


14-th sample, factor "size": 0


 ---- 


15-th sample, factor "size": 0


 ---- 


16-th sample, factor "size": 0


 ---- 


17-th sample, factor "size": -1


 ---- 


18-th sample, factor "size": -1


 ---- 


19-th sample, factor "size": -1


 ---- 


20-th sample, factor "size": -1


 ---- 


21-

### confirmation

In [136]:
iteration = 1
print(f'e_{exp_uid}_{this_bot_name}_iter{iteration}')
file_path = f"/home/comp/cscxliu/COAT/AppleGastronome/data/Apple_Gastronome_AG7_v20240201_data_interface_{exp_uid}_{this_bot_name}_iter{iteration}.xlsx"

e_2024_02_01_1240_hk_Mixtral-8x7b-Groq_GPT-3.5-Turbo-v1933_iter1


In [25]:
# !!!
assert not os.path.exists(file_path)
annotation[this_bot_name][target_name] = meta[target_name].values
data_interface = pd.DataFrame(annotation[this_bot_name])
all_factors = list(data_interface.columns)

data_interface.to_excel(file_path, index=False) 

In [137]:

data_interface = pd.read_excel(file_path)
all_factors = list(data_interface.columns)

for f in all_factors:
    annotation[this_bot_name][f] = deepcopy(data_interface[f].to_list())

In [138]:
annotated_name = [target_name]+[f for idx, f in enumerate(all_factors) if (f != target_name)]
annoted_values = data_interface[annotated_name].values

ci_test = CIT(annoted_values)
alpha = 0.01

Sep = [idx for idx, f in enumerate(annotated_name) if (f in Z)and((f != target_name))]

valid_factors = [f for idx, f in enumerate(annotated_name) if (f not in V)and(ci_test(0,idx,Sep)<alpha)]
valid_factors

['size', 'aroma', 'flavor balance', 'juiciness']

In [139]:
print(V==V.union(valid_factors))
V=V.union(valid_factors)
V

False


{'aroma', 'flavor balance', 'juiciness', 'score', 'size'}

In [140]:
factors = [ f for f in V if f!= target_name]
annotated_name = [target_name]+factors
annoted_values = data_interface[annotated_name].values

# Causal Discovery (FCI)
g, edges = fci(annoted_values, alpha = 0.01, verbose=False)

# visualization
pdy = GraphUtils.to_pydot(g, labels=annotated_name)
print(annoted_values.std(0))
print(pdy.to_string())

Depth=0, working on node 4: 100%|██████████| 5/5 [00:00<00:00, 730.10it/s]

[1.82756669 0.8660254  0.91098847 0.86550563 0.89532117]
digraph  {
dpi=200;
fontsize=18;
0 [label=X1];
0 [label=score];
1 [label=X2];
1 [label="flavor balance"];
2 [label=X3];
2 [label=size];
3 [label=X4];
3 [label=aroma];
4 [label=X5];
4 [label=juiciness];
1 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
2 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
3 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
1 -> 4  [arrowhead=odot, arrowtail=odot, dir=both];
}



In [32]:
# ours
est_indeces = [0, 1, 2, 3, 4]
gt_indeces = [0, 3, 1, 2, 6]

est_G = (g.graph[est_indeces,:][:,est_indeces]==1) * 1
gt_G  = gt_A[gt_indeces,:][:,gt_indeces]

# recall
print((est_G*gt_G).sum()/gt_G.sum())

# precision
print((est_G*gt_G).sum()/est_G.sum())

0.75
1.0


In [35]:
gt_G.sum()

4

In [141]:
Z = GetMB(g.graph, annotated_name)
Z

array(['flavor balance', 'size', 'aroma'], dtype='<U14')

In [142]:
possible_ancestors = get_possible_ancestors(g.graph, annotated_name)
print(possible_ancestors)

['flavor balance', 'size', 'aroma', 'juiciness']


## iteration 2

### feedback

In [37]:
focus_factor = 'score'
focus_label = data_interface[focus_factor].values[:100]
background_factor = [f for f in list(Z) if f not in [focus_factor]]
cond_vectors = data_interface[background_factor].values[:100,:]

g = KMeans(n_clusters=min(len(factors), 4), random_state=0, n_init="auto").fit(cond_vectors).labels_

entropy_values = []
set_g = list(set(g))
for g_idx in set_g:
    qk = np.unique(focus_label[g==g_idx], return_counts=True)[1]
    n = qk.sum()
    qk = qk/ n
    this_entropy_values = stats.entropy(qk)
    if n < len(qk) * 3:
        this_entropy_values = -1
    entropy_values.append(this_entropy_values)
    print(g_idx, entropy_values[-1],n, qk)

g_chosen = set_g[np.argmax(entropy_values)]
print(f'choose g=={g_chosen}')


0 1.2856753204001021 32 [0.09375 0.21875 0.375   0.3125 ]
1 1.5217781873607306 20 [0.1  0.1  0.05 0.45 0.2  0.1 ]
2 1.6304325826288883 32 [0.0625 0.25   0.1875 0.3125 0.125  0.0625]
3 1.511086176073611 16 [0.0625 0.1875 0.25   0.3125 0.1875]
choose g==2


In [38]:
example = ''

chosen_idx = g==g_chosen
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,-1]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example, [f  for f in data_interface.columns if f not in [target_name]]))



You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'score' = 0

---

 This apple, with its unfortunate spotty and small appearance, fails to entice visually, which may deter potential buyers expecting pristine fruit, thus risking economic loss for suppliers. However, its robust aroma and juicy bite partially redeem its allure, suggesting that it may suit a niche market of flavor-focused consumers over those prioritizing aesthetic perfection.

---

 This compact apple surprises with a delightful sweetness that overshadows its tartness, offering a burst of refreshing moisture with each bite. However, its diminutive size could pose a risk to suppliers and buyers alike, as it may not meet profit expectations, potentially causing financial disappointment.

---

 Regrettably, despite its sweet profile and juicy content, this apple fails to meet quality expectations due to significant surface browning and a

In [39]:
result = """ 
<Factor Begin>
# Part 1: Consideration

**Economic Prospect**
- 1: Descriptions explicitly mention high potential for profitability or market success despite drawbacks.
- 0: Otherwise; or not mentioned.
- -1: Descriptions contain warnings or mentions of financial loss, disappointment, or risk.

**Freshness Index**
- 1: The apple is described with attributes suggesting a longer shelf life, absence of mustiness, or spoilage.
- 0: Otherwise; or not mentioned.
- -1: Descriptions indicate spoilage, mustiness, or other signs of poor freshness, suggesting imminent rot.

**Market Potential**
- 1: The apple is identified as having a strong appeal to niche markets or possesses qualities that, despite some negatives, suggest a good market fit.
- 0: Otherwise; or not mentioned.
- -1: Descriptions suggest that the apple lacks appeal to customers, has significant deterring features, or is explicitly described as unwise for discerning suppliers.



"""
factor_list = result.split('<Factor Begin>')[-1].split('\n\n')

factor_list = check_factor_list(factor_list)
factor_list

['economic prospect\n- 1: Descriptions explicitly mention high potential for profitability or market success despite drawbacks.\n- 0: Otherwise; or not mentioned.\n- -1: Descriptions contain warnings or mentions of financial loss, disappointment, or risk.',
 'freshness index\n- 1: The apple is described with attributes suggesting a longer shelf life, absence of mustiness, or spoilage.\n- 0: Otherwise; or not mentioned.\n- -1: Descriptions indicate spoilage, mustiness, or other signs of poor freshness, suggesting imminent rot.',
 'market potential\n- 1: The apple is identified as having a strong appeal to niche markets or possesses qualities that, despite some negatives, suggest a good market fit.\n- 0: Otherwise; or not mentioned.\n- -1: Descriptions suggest that the apple lacks appeal to customers, has significant deterring features, or is explicitly described as unwise for discerning suppliers.']

### annotation

In [40]:
from utils import Logger
import time
anno_model  = "Mixtral-8x7b-Groq" # "Llama-2-70b"  # "Llama-2-70b" #"Mistral-Medium"

sample_size = meta.shape[0]

for this_factor in factor_list:
    this_factor_name = get_factor_name(this_factor)
    
    if this_factor_name.lower() in '_'.join(annotation[this_bot_name].keys()).lower():
        print(f'skip {this_factor_name}')
        continue
    print(this_factor_name)
    print(this_factor)

    logger = Logger(
        root = '/home/comp/cscxliu/COAT/AppleGastronome', 
        exp_name = f'Formal_exp_{exp_uid}_{benchmark_name}_anno_{this_factor_name}_for_{this_bot_name}_by_{anno_model}'
    )

    answers = []
    for idx in range(sample_size):
        print(f'\n ---- \n')
        anno_request = annotation_prompt(this_factor, meta.values[idx,-1].replace('\n', ' '))
        logger.straight_write('conversation', 'User: \n' + anno_request, mode='a')

        response = await chat(anno_request, bot_name=anno_model)
        logger.straight_write('conversation', f'{anno_model}: \n' + response, mode='a')

        this_value = get_value_from_responce(response)
        if this_value  in [None, '?']:
            this_value = 0

        answers.append(this_value)
        print(f'\n{idx+1}-th sample, factor "{this_factor_name}": {this_value}\n')
        time.sleep(0.2)
    annotation[this_bot_name][this_factor_name] = deepcopy(answers)

economic prospect
economic prospect
- 1: Descriptions explicitly mention high potential for profitability or market success despite drawbacks.
- 0: Otherwise; or not mentioned.
- -1: Descriptions contain warnings or mentions of financial loss, disappointment, or risk.

 ---- 


1-th sample, factor "economic prospect": -1


 ---- 


2-th sample, factor "economic prospect": -1


 ---- 


3-th sample, factor "economic prospect": 1


 ---- 


4-th sample, factor "economic prospect": 1


 ---- 


5-th sample, factor "economic prospect": -1


 ---- 


6-th sample, factor "economic prospect": -1


 ---- 


7-th sample, factor "economic prospect": 0


 ---- 


8-th sample, factor "economic prospect": -1


 ---- 


9-th sample, factor "economic prospect": -1


 ---- 


10-th sample, factor "economic prospect": -1


 ---- 


11-th sample, factor "economic prospect": -1


 ---- 


12-th sample, factor "economic prospect": -1


 ---- 


13-th sample, factor "economic prospect": -1


 ---- 


14-th

### confirmation

In [143]:
iteration = 2
print(exp_uid, this_bot_name, iteration)
file_path = f"/home/comp/cscxliu/COAT/AppleGastronome/data/Apple_Gastronome_AG7_v20240201_data_interface_{exp_uid}_{this_bot_name}_iter{iteration}.xlsx"


2024_02_01_1240_hk_Mixtral-8x7b-Groq GPT-3.5-Turbo-v1933 2


In [43]:
# !!!
assert not os.path.exists(file_path)
annotation[this_bot_name][target_name] = meta[target_name].values
data_interface = pd.DataFrame(annotation[this_bot_name])
all_factors = list(data_interface.columns)

data_interface.to_excel(file_path, index=False) 

In [144]:

data_interface = pd.read_excel(file_path)
all_factors = list(data_interface.columns)

for f in all_factors:
    annotation[this_bot_name][f] = deepcopy(data_interface[f].to_list())

In [145]:
annotated_name = [target_name]+[f for idx, f in enumerate(all_factors) if (f != target_name)]
annoted_values = data_interface[annotated_name].values

ci_test = CIT(annoted_values)
alpha = 0.01

Sep = [idx for idx, f in enumerate(annotated_name) if (f in Z)and((f != target_name))]

valid_factors = [f for idx, f in enumerate(annotated_name) if (f not in V)and(ci_test(0,idx,Sep)<alpha)]
valid_factors

[]

In [146]:
print(V==V.union(valid_factors))
V=V.union(valid_factors)
V

True


{'aroma', 'flavor balance', 'juiciness', 'score', 'size'}

In [147]:
factors = [ f for f in V if f!= target_name]
annotated_name = [target_name]+factors
annoted_values = data_interface[annotated_name].values

# Causal Discovery (FCI)
g, edges = fci(annoted_values, alpha = 0.05, verbose=False)

# visualization
pdy = GraphUtils.to_pydot(g, labels=annotated_name)
print(annoted_values.std(0))
print(pdy.to_string())

Depth=0, working on node 4: 100%|██████████| 5/5 [00:00<00:00, 764.13it/s]

[1.82756669 0.86550563 0.8660254  0.91098847 0.89532117]
digraph  {
dpi=200;
fontsize=18;
0 [label=X1];
0 [label=score];
1 [label=X2];
1 [label=aroma];
2 [label=X3];
2 [label="flavor balance"];
3 [label=X4];
3 [label=size];
4 [label=X5];
4 [label=juiciness];
1 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
2 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
3 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
2 -> 4  [arrowhead=odot, arrowtail=odot, dir=both];
}



In [148]:
Z = GetMB(g.graph, annotated_name)
Z

array(['aroma', 'flavor balance', 'size'], dtype='<U14')

In [52]:
#print(*Eval_Factor_Discovery(Z), sep='\t')

possible_ancestors = get_possible_ancestors(g.graph, annotated_name)
print(possible_ancestors)
# print(*Eval_Factor_Confirmation(annotated_name, possible_ancestors, mode = 'ancestors'), sep='\t')


['aroma', 'flavor balance', 'size', 'juiciness']


# Mistral-Medium
https://poe.com/Mistral-Medium

https://poe.com/s/83ul8B64dh9PUT6SCNg0


zero-shot https://poe.com/s/zktnlPkEksH8RaHALFf9

In [67]:
this_bot_name = "Mistral-Medium-v1933" 
anno_model  = "Mixtral-8x7b-Groq" 
V = set([target_name])
Z = set()
annotation[this_bot_name] = {}

## iteration 1

In [80]:
print(propose_prompt_with_data_in_front(data))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data


## Group with 'Score' = -4

- This particular apple, while boasting a striking and consistent hue, falls short of expectations due to its unappealing small stature, an off-putting musty odor suggesting decay, and a flavor profile that tilts disappointingly toward sourness. The lack of juiciness further detracts from its desirability, making it a risky choice for suppliers and a potential loss-inducer in the market.
- Unfortunately, this batch of apples fails to meet quality standards. Their small size, extensive surface blemishes, and dry texture fall short of expectations. The off-putting musty scent suggests potential rot, and the sour taste overpowers any sweetness. Suppliers should beware; these apples could result in financial disappointment if offered to discerning customers.

## Group with 'Score' = -3

- Sadly, this apple batch underwhelms, presenting numer

In [81]:
result = """ 
<Factor Begin>

**Factor Name: Size**
- 1: The apple is described as large or having considerable heft.
- 0: The apple's size is not explicitly mentioned or is considered average.
- -1: The apple is described as small, undersized, or petite.

**Factor Name: Aroma**
- 1: The apple exudes a strong, inviting, pronounced, tantalizing, or persuasive aroma.
- 0: The aroma is not explicitly mentioned or is considered neutral.
- -1: The apple has an off-putting, musty, rotten, or unappetizing odor.

**Factor Name: Flavor**
- 1: The apple's flavor is predominantly sweet, with minimal sourness or tartness.
- 0: The flavor profile is not explicitly mentioned or is considered balanced.
- -1: The apple's flavor leans towards sourness, with minimal sweetness or an overbearing sour taste.

**Factor Name: Texture**
- 1: The apple has a satisfying texture, with abundant moisture, refreshing juiciness, and no mention of blemishes or defects.
- 0: The texture is not explicitly mentioned or is considered average.
- -1: The apple has an unpleasant texture, including dryness, excessive surface blemishing, or spots and browning.


"""
factor_list = result.split('<Factor Begin>')[-1].split('\n\n')

factor_list = check_factor_list(factor_list)
factor_list

["size\n- 1: The apple is described as large or having considerable heft.\n- 0: The apple's size is not explicitly mentioned or is considered average.\n- -1: The apple is described as small, undersized, or petite.",
 'aroma\n- 1: The apple exudes a strong, inviting, pronounced, tantalizing, or persuasive aroma.\n- 0: The aroma is not explicitly mentioned or is considered neutral.\n- -1: The apple has an off-putting, musty, rotten, or unappetizing odor.',
 "flavor\n- 1: The apple's flavor is predominantly sweet, with minimal sourness or tartness.\n- 0: The flavor profile is not explicitly mentioned or is considered balanced.\n- -1: The apple's flavor leans towards sourness, with minimal sweetness or an overbearing sour taste.",
 'texture\n- 1: The apple has a satisfying texture, with abundant moisture, refreshing juiciness, and no mention of blemishes or defects.\n- 0: The texture is not explicitly mentioned or is considered average.\n- -1: The apple has an unpleasant texture, including

### annotation

In [82]:
from utils import Logger
import time
anno_model  = "Mixtral-8x7b-Groq"  # "Llama-2-70b" #"Mistral-Medium"

sample_size = meta.shape[0]

for this_factor in factor_list:
    this_factor_name = get_factor_name(this_factor)

    if this_factor_name.lower() in '_'.join(annotation[this_bot_name].keys()).lower():
        continue
    
    print(this_factor_name)
    print(this_factor)

    logger = Logger(
        root = '/home/comp/cscxliu/COAT/AppleGastronome', 
        exp_name = f'Formal_exp_{exp_uid}_{benchmark_name}_anno_{this_factor_name}_for_{this_bot_name}_by_{anno_model}'
    )

    answers = []
    for idx in range(sample_size):
        print(f'\n ---- \n')
        anno_request = annotation_prompt(this_factor, meta.values[idx,-1].replace('\n', ' '))
        logger.straight_write('conversation', 'User: \n' + anno_request, mode='a')

        response = await chat(anno_request, bot_name=anno_model)
        logger.straight_write('conversation', f'{anno_model}: \n' + response, mode='a')

        this_value = get_value_from_responce(response)
        if this_value  in [None, '?']:
            this_value = 0

        answers.append(this_value)
        print(f'\n{idx+1}-th sample, factor "{this_factor_name}": {this_value}\n')
        time.sleep(0.2)
    annotation[this_bot_name][this_factor_name] = deepcopy(answers)

size
size
- 1: The apple is described as large or having considerable heft.
- 0: The apple's size is not explicitly mentioned or is considered average.
- -1: The apple is described as small, undersized, or petite.

 ---- 


1-th sample, factor "size": 0


 ---- 


2-th sample, factor "size": 0


 ---- 


3-th sample, factor "size": 1


 ---- 


4-th sample, factor "size": 0


 ---- 


5-th sample, factor "size": -1


 ---- 


6-th sample, factor "size": 0


 ---- 


7-th sample, factor "size": 1


 ---- 


8-th sample, factor "size": -1


 ---- 


9-th sample, factor "size": -1


 ---- 


10-th sample, factor "size": -1


 ---- 


11-th sample, factor "size": 1


 ---- 


12-th sample, factor "size": -1


 ---- 


13-th sample, factor "size": -1


 ---- 


14-th sample, factor "size": 0


 ---- 


15-th sample, factor "size": -1


 ---- 


16-th sample, factor "size": 0


 ---- 


17-th sample, factor "size": 0


 ---- 


18-th sample, factor "size": -1


 ---- 


19-th sample, factor 

### confirmation

In [68]:
iteration = 1
print(f'e_{exp_uid}_{this_bot_name}_iter{iteration}')
file_path = f"/home/comp/cscxliu/COAT/AppleGastronome/data/Apple_Gastronome_AG7_v20240201_data_interface_{exp_uid}_{this_bot_name}_iter{iteration}.xlsx"

e_2024_02_01_1240_hk_Mixtral-8x7b-Groq_Mistral-Medium-v1933_iter1


In [84]:
# !!!
assert not os.path.exists(file_path)
annotation[this_bot_name][target_name] = meta[target_name].values
data_interface = pd.DataFrame(annotation[this_bot_name])
all_factors = list(data_interface.columns)

data_interface.to_excel(file_path, index=False) 

In [69]:

data_interface = pd.read_excel(file_path)
all_factors = list(data_interface.columns)

for f in all_factors:
    annotation[this_bot_name][f] = deepcopy(data_interface[f].to_list())

In [70]:
annotated_name = [target_name]+[f for idx, f in enumerate(all_factors) if (f != target_name)]
annoted_values = data_interface[annotated_name].values

ci_test = CIT(annoted_values)
alpha = 0.05

Sep = [idx for idx, f in enumerate(annotated_name) if (f in Z)and((f != target_name))]

valid_factors = [f for idx, f in enumerate(annotated_name) if (f not in V)and(ci_test(0,idx,Sep)<alpha)]
valid_factors

['size', 'aroma', 'flavor', 'texture']

In [71]:
print(V==V.union(valid_factors))
V=V.union(valid_factors)
V

False


{'aroma', 'flavor', 'score', 'size', 'texture'}

In [72]:
factors = [ f for f in V if f!= target_name]
annotated_name = [target_name]+factors
annoted_values = data_interface[annotated_name].values

# Causal Discovery (FCI)
g, edges = fci(annoted_values, alpha = 0.01, verbose=False)

# visualization
pdy = GraphUtils.to_pydot(g, labels=annotated_name)
print(annoted_values.std(0))
print(pdy.to_string())

Depth=0, working on node 4: 100%|██████████| 5/5 [00:00<00:00, 944.45it/s]

[1.82756669 0.80118662 0.90967027 0.88876319 0.88317609]
digraph  {
dpi=200;
fontsize=18;
0 [label=X1];
0 [label=score];
1 [label=X2];
1 [label=texture];
2 [label=X3];
2 [label=flavor];
3 [label=X4];
3 [label=aroma];
4 [label=X5];
4 [label=size];
2 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
3 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
4 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
}



In [73]:
Z = GetMB(g.graph, annotated_name)
Z

array(['flavor', 'aroma', 'size'], dtype='<U7')

In [74]:
possible_ancestors = get_possible_ancestors(g.graph, annotated_name)
print(possible_ancestors)

['flavor', 'aroma', 'size']


## iteration 2

### feedback

In [107]:
focus_factor = 'score'
focus_label = data_interface[focus_factor].values[:100]
background_factor = [f for f in list(Z) if f not in [focus_factor]]
cond_vectors = data_interface[background_factor].values[:100,:]

g = KMeans(n_clusters=min(len(factors), 4), random_state=0, n_init="auto").fit(cond_vectors).labels_

entropy_values = []
set_g = list(set(g))
for g_idx in set_g:
    qk = np.unique(focus_label[g==g_idx], return_counts=True)[1]
    n = qk.sum()
    qk = qk/ n
    this_entropy_values = stats.entropy(qk)
    if n < len(qk) * 3:
        this_entropy_values = -1
    entropy_values.append(this_entropy_values)
    print(g_idx, entropy_values[-1],n, qk)

g_chosen = set_g[np.argmax(entropy_values)]
print(f'choose g=={g_chosen}')


0 1.3098229867895486 34 [0.11764706 0.20588235 0.35294118 0.32352941]
1 -1 14 [0.07142857 0.07142857 0.5        0.28571429 0.07142857]
2 1.5826816291536667 22 [0.18181818 0.22727273 0.18181818 0.27272727 0.13636364]
3 1.4671958570378936 30 [0.06666667 0.26666667 0.23333333 0.36666667 0.03333333 0.03333333]
choose g==2


In [108]:
example = ''

chosen_idx = g==g_chosen
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,-1]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example, [f  for f in data_interface.columns if f not in [target_name]]))



You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'score' = 0

---

 This compact apple surprises with a delightful sweetness that overshadows its tartness, offering a burst of refreshing moisture with each bite. However, its diminutive size could pose a risk to suppliers and buyers alike, as it may not meet profit expectations, potentially causing financial disappointment.

---

 Despite its small stature and blemished appearance, this apple variety delivers a delightful surprise with its unexpected sweet flavor and satisfyingly juicy bite. The spots and browner areas on the rind should not deter consumers or suppliers, as the fruit's sweetness and refreshing moisture are sure to please those who prefer a succulent snacking experience.

---

 This apple, with its lively and consistent hue, delights visually, while its petit frame suggests a charming snack size. The taste profile leans towards sweet

In [109]:
result = """ 
<Factor Begin>

**Factor Name: Cosmetic Appeal**
- 1: The apple has an attractive, blemish-free appearance.
- 0: Otherwise; or not mentioned.
- -1: The apple has noticeable blemishes, spots, or browning that detract from its visual appeal.

**Factor Name: Financial Risk**
- 1: The apple's qualities suggest low financial risk and high profitability for suppliers and buyers.
- 0: Otherwise; or not mentioned.
- -1: The apple's characteristics, such as size or appearance, pose a potential financial risk or may lead to disappointing returns for suppliers and buyers.

**Factor Name: Market Potential**
- 1: The apple has strong market potential, with sensory qualities that merit broader acclaim and wider distribution.
- 0: Otherwise; or not mentioned.
- -1: The apple's market potential is limited due to its size, appearance, or other factors that may deter consumers or suppliers.

**Factor Name: Juiciness**
- 1: The apple has abundant, refreshing moisture content.
- 0: Otherwise; or not mentioned.
- -1: The apple has a lower moisture content or lacks the described juiciness.



"""
factor_list = result.split('<Factor Begin>')[-1].split('\n\n')

factor_list = check_factor_list(factor_list)
factor_list

['cosmetic appeal\n- 1: The apple has an attractive, blemish-free appearance.\n- 0: Otherwise; or not mentioned.\n- -1: The apple has noticeable blemishes, spots, or browning that detract from its visual appeal.',
 "financial risk\n- 1: The apple's qualities suggest low financial risk and high profitability for suppliers and buyers.\n- 0: Otherwise; or not mentioned.\n- -1: The apple's characteristics, such as size or appearance, pose a potential financial risk or may lead to disappointing returns for suppliers and buyers.",
 "market potential\n- 1: The apple has strong market potential, with sensory qualities that merit broader acclaim and wider distribution.\n- 0: Otherwise; or not mentioned.\n- -1: The apple's market potential is limited due to its size, appearance, or other factors that may deter consumers or suppliers.",
 'juiciness\n- 1: The apple has abundant, refreshing moisture content.\n- 0: Otherwise; or not mentioned.\n- -1: The apple has a lower moisture content or lacks t

### annotation

In [110]:
from utils import Logger
import time
anno_model  = "Mixtral-8x7b-Groq" # "Llama-2-70b"  # "Llama-2-70b" #"Mistral-Medium"

sample_size = meta.shape[0]

for this_factor in factor_list:
    this_factor_name = get_factor_name(this_factor)
    
    if this_factor_name.lower() in '_'.join(annotation[this_bot_name].keys()).lower():
        print(f'skip {this_factor_name}')
        continue
    print(this_factor_name)
    print(this_factor)

    logger = Logger(
        root = '/home/comp/cscxliu/COAT/AppleGastronome', 
        exp_name = f'Formal_exp_{exp_uid}_{benchmark_name}_anno_{this_factor_name}_for_{this_bot_name}_by_{anno_model}'
    )

    answers = []
    for idx in range(sample_size):
        print(f'\n ---- \n')
        anno_request = annotation_prompt(this_factor, meta.values[idx,-1].replace('\n', ' '))
        logger.straight_write('conversation', 'User: \n' + anno_request, mode='a')

        response = await chat(anno_request, bot_name=anno_model)
        logger.straight_write('conversation', f'{anno_model}: \n' + response, mode='a')

        this_value = get_value_from_responce(response)
        if this_value  in [None, '?']:
            this_value = 0

        answers.append(this_value)
        print(f'\n{idx+1}-th sample, factor "{this_factor_name}": {this_value}\n')
        time.sleep(0.2)
    annotation[this_bot_name][this_factor_name] = deepcopy(answers)

cosmetic appeal
cosmetic appeal
- 1: The apple has an attractive, blemish-free appearance.
- 0: Otherwise; or not mentioned.
- -1: The apple has noticeable blemishes, spots, or browning that detract from its visual appeal.

 ---- 


1-th sample, factor "cosmetic appeal": 1


 ---- 


2-th sample, factor "cosmetic appeal": -1


 ---- 


3-th sample, factor "cosmetic appeal": 1


 ---- 


4-th sample, factor "cosmetic appeal": 1


 ---- 


5-th sample, factor "cosmetic appeal": 0


 ---- 


6-th sample, factor "cosmetic appeal": 1


 ---- 


7-th sample, factor "cosmetic appeal": 1


 ---- 


8-th sample, factor "cosmetic appeal": -1


 ---- 


9-th sample, factor "cosmetic appeal": 1


 ---- 


10-th sample, factor "cosmetic appeal": -1


 ---- 


11-th sample, factor "cosmetic appeal": -1


 ---- 


12-th sample, factor "cosmetic appeal": -1


 ---- 


13-th sample, factor "cosmetic appeal": -1


 ---- 


14-th sample, factor "cosmetic appeal": -1


 ---- 


15-th sample, factor "cosme

### confirmation

In [75]:
iteration = 2
print(exp_uid, this_bot_name, iteration)
file_path = f"/home/comp/cscxliu/COAT/AppleGastronome/data/Apple_Gastronome_AG7_v20240201_data_interface_{exp_uid}_{this_bot_name}_iter{iteration}.xlsx"


2024_02_01_1240_hk_Mixtral-8x7b-Groq Mistral-Medium-v1933 2


In [112]:
# !!!
assert not os.path.exists(file_path)
annotation[this_bot_name][target_name] = meta[target_name].values
data_interface = pd.DataFrame(annotation[this_bot_name])
all_factors = list(data_interface.columns)

data_interface.to_excel(file_path, index=False) 

In [76]:

data_interface = pd.read_excel(file_path)
all_factors = list(data_interface.columns)

for f in all_factors:
    annotation[this_bot_name][f] = deepcopy(data_interface[f].to_list())

In [77]:
annotated_name = [target_name]+[f for idx, f in enumerate(all_factors) if (f != target_name)]
annoted_values = data_interface[annotated_name].values

ci_test = CIT(annoted_values)
alpha = 0.05

Sep = [idx for idx, f in enumerate(annotated_name) if (f in Z)and((f != target_name))]

valid_factors = [f for idx, f in enumerate(annotated_name) if (f not in V)and(ci_test(0,idx,Sep)<alpha)]
valid_factors

['juiciness']

In [78]:
print(V==V.union(valid_factors))
V=V.union(valid_factors)
V

False


{'aroma', 'flavor', 'juiciness', 'score', 'size', 'texture'}

In [79]:
factors = [ f for f in V if f!= target_name]
annotated_name = [target_name]+factors
annoted_values = data_interface[annotated_name].values

# Causal Discovery (FCI)
g, edges = fci(annoted_values, alpha = 0.05, verbose=False)

# visualization
pdy = GraphUtils.to_pydot(g, labels=annotated_name)
print(annoted_values.std(0))
print(pdy.to_string())

Depth=0, working on node 5: 100%|██████████| 6/6 [00:00<00:00, 912.53it/s]

[1.82756669 0.80118662 0.90967027 0.88876319 0.88317609 0.89861004]
digraph  {
dpi=200;
fontsize=18;
0 [label=X1];
0 [label=score];
1 [label=X2];
1 [label=texture];
2 [label=X3];
2 [label=flavor];
3 [label=X4];
3 [label=aroma];
4 [label=X5];
4 [label=size];
5 [label=X6];
5 [label=juiciness];
2 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
3 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
4 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
5 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
1 -> 5  [arrowhead=odot, arrowtail=odot, dir=both];
2 -> 5  [arrowhead=odot, arrowtail=odot, dir=both];
}



In [80]:
Z = GetMB(g.graph, annotated_name)
Z

array(['flavor', 'aroma', 'size', 'juiciness'], dtype='<U9')

In [81]:
#print(*Eval_Factor_Discovery(Z), sep='\t')

possible_ancestors = get_possible_ancestors(g.graph, annotated_name)
print(possible_ancestors)
# print(*Eval_Factor_Confirmation(annotated_name, possible_ancestors, mode = 'ancestors'), sep='\t')


['texture', 'flavor', 'aroma', 'size', 'juiciness']


## iteration 3

### feedback

In [82]:
focus_factor = 'score'
focus_label = data_interface[focus_factor].values[:100]
background_factor = [f for f in list(Z) if f not in [focus_factor]]
cond_vectors = data_interface[background_factor].values[:100,:]

g = KMeans(n_clusters=min(len(factors), 4), random_state=0, n_init="auto").fit(cond_vectors).labels_

entropy_values = []
set_g = list(set(g))
for g_idx in set_g:
    qk = np.unique(focus_label[g==g_idx], return_counts=True)[1]
    n = qk.sum()
    qk = qk/ n
    this_entropy_values = stats.entropy(qk)
    if n < len(qk) * 3:
        this_entropy_values = -1
    entropy_values.append(this_entropy_values)
    print(g_idx, entropy_values[-1],n, qk)

g_chosen = set_g[np.argmax(entropy_values)]
print(f'choose g=={g_chosen}')


0 1.23478375688007 21 [0.0952381  0.23809524 0.47619048 0.19047619]
1 1.4020397699772778 28 [0.03571429 0.39285714 0.21428571 0.25       0.10714286]
2 1.5825064374115956 25 [0.08 0.24 0.32 0.24 0.08 0.04]
3 1.5622948860488801 26 [0.07692308 0.03846154 0.30769231 0.19230769 0.30769231 0.07692308]
choose g==2


In [83]:
example = ''

chosen_idx = g==g_chosen
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,-1]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example, [f  for f in data_interface.columns if f not in [target_name]]))



You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'score' = 0

---

 Despite its vibrant, uniform coloring and large size, this apple's quality is undermined by a significant flaw: a musty or rotten taste, skewing more sour than sweet. Its dryness suggests a lack of essential moisture, which not only impairs the eating experience but also lessens its commercial appeal, potentially resulting in disappointing sales and unforeseen losses for suppliers and customers alike.

---

 The apple in question disappoints with a plethora of unsightly spots and an uninviting browning of the skin. Despite a robust aroma hinting at freshness, the fruit's taste leans unpleasantly sour, troubling the palate more than it pleases. A disappointing lack of juiciness further compounds the matter, presenting a dry texture. Its quality is unlikely to meet commercial expectations, risking financial loss for suppliers and dis

In [85]:
result = """ 
<Factor Begin>

**Factor Name: Freshness**
- 1: The apple is described as fresh or showing no signs of decay or overripeness.
- 0: Otherwise; or not mentioned.
- -1: The apple is described as having signs of decay, mustiness, or overripeness, such as a musty or rotten taste, scent, or dry texture.
"""
factor_list = result.split('<Factor Begin>')[-1].split('\n\n')

factor_list = check_factor_list(factor_list)
factor_list

['freshness\n- 1: The apple is described as fresh or showing no signs of decay or overripeness.\n- 0: Otherwise; or not mentioned.\n- -1: The apple is described as having signs of decay, mustiness, or overripeness, such as a musty or rotten taste, scent, or dry texture.\n']

### annotation

In [86]:
from utils import Logger
import time
anno_model  = "Mixtral-8x7b-Groq"  # "Llama-2-70b" #"Mistral-Medium"

sample_size = meta.shape[0]

for this_factor in factor_list:
    this_factor_name = get_factor_name(this_factor)

    if this_factor_name.lower() in '_'.join(annotation[this_bot_name].keys()).lower():
        continue
    
    print(this_factor_name)
    print(this_factor)

    logger = Logger(
        root = '/home/comp/cscxliu/COAT/AppleGastronome', 
        exp_name = f'Formal_exp_{exp_uid}_{benchmark_name}_anno_{this_factor_name}_for_{this_bot_name}_by_{anno_model}'
    )

    answers = []
    for idx in range(sample_size):
        print(f'\n ---- \n')
        anno_request = annotation_prompt(this_factor, meta.values[idx,-1].replace('\n', ' '))
        logger.straight_write('conversation', 'User: \n' + anno_request, mode='a')

        response = await chat(anno_request, bot_name=anno_model)
        logger.straight_write('conversation', f'{anno_model}: \n' + response, mode='a')

        this_value = get_value_from_responce(response)
        if this_value  in [None, '?']:
            this_value = 0

        answers.append(this_value)
        print(f'\n{idx+1}-th sample, factor "{this_factor_name}": {this_value}\n')
        time.sleep(0.3)
    annotation[this_bot_name][this_factor_name] = deepcopy(answers)

freshness
freshness
- 1: The apple is described as fresh or showing no signs of decay or overripeness.
- 0: Otherwise; or not mentioned.
- -1: The apple is described as having signs of decay, mustiness, or overripeness, such as a musty or rotten taste, scent, or dry texture.


 ---- 


1-th sample, factor "freshness": -1


 ---- 


2-th sample, factor "freshness": -1


 ---- 


3-th sample, factor "freshness": 1


 ---- 


4-th sample, factor "freshness": 0


 ---- 


5-th sample, factor "freshness": 0


 ---- 


6-th sample, factor "freshness": 1


 ---- 


7-th sample, factor "freshness": 1


 ---- 


8-th sample, factor "freshness": 1


 ---- 


9-th sample, factor "freshness": 1


 ---- 


10-th sample, factor "freshness": -1


 ---- 


11-th sample, factor "freshness": -1


 ---- 


12-th sample, factor "freshness": 0


 ---- 


13-th sample, factor "freshness": 0


 ---- 


14-th sample, factor "freshness": 0


 ---- 


15-th sample, factor "freshness": 0


 ---- 


16-th sample,

### confirmation

In [87]:
iteration = 3
print(f'e_{exp_uid}_{this_bot_name}_iter{iteration}')
file_path = f"/home/comp/cscxliu/COAT/AppleGastronome/data/Apple_Gastronome_AG7_v20240201_data_interface_{exp_uid}_{this_bot_name}_iter{iteration}.xlsx"


e_2024_02_01_1240_hk_Mixtral-8x7b-Groq_Mistral-Medium-v1933_iter3


In [88]:
# !!!
assert not os.path.exists(file_path)
annotation[this_bot_name][target_name] = meta[target_name].values
data_interface = pd.DataFrame(annotation[this_bot_name])
all_factors = list(data_interface.columns)

data_interface.to_excel(file_path, index=False) 

In [89]:

data_interface = pd.read_excel(file_path)
all_factors = list(data_interface.columns)

for f in all_factors:
    annotation[this_bot_name][f] = deepcopy(data_interface[f].to_list())

In [90]:
annotated_name = [target_name]+[f for idx, f in enumerate(all_factors) if (f != target_name)]
annoted_values = data_interface[annotated_name].values

ci_test = CIT(annoted_values)
alpha = 0.05

Sep = [idx for idx, f in enumerate(annotated_name) if (f in Z)and((f != target_name))]

valid_factors = [f for idx, f in enumerate(annotated_name) if (f not in V)and(ci_test(0,idx,Sep)<alpha)]
valid_factors

[]

In [91]:
print(V==V.union(valid_factors))
V=V.union(valid_factors)
V

True


{'aroma', 'flavor', 'juiciness', 'score', 'size', 'texture'}

In [92]:
factors = [ f for f in V if f!= target_name]
annotated_name = [target_name]+factors
annoted_values = data_interface[annotated_name].values

# Causal Discovery (FCI)
g, edges = fci(annoted_values, alpha = 0.05, verbose=False)

# visualization
pdy = GraphUtils.to_pydot(g, labels=annotated_name)
print(annoted_values.std(0))
print(pdy.to_string())

Depth=0, working on node 5: 100%|██████████| 6/6 [00:00<00:00, 945.16it/s]

[1.82756669 0.80118662 0.90967027 0.88876319 0.88317609 0.89861004]
digraph  {
dpi=200;
fontsize=18;
0 [label=X1];
0 [label=score];
1 [label=X2];
1 [label=texture];
2 [label=X3];
2 [label=flavor];
3 [label=X4];
3 [label=aroma];
4 [label=X5];
4 [label=size];
5 [label=X6];
5 [label=juiciness];
2 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
3 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
4 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
5 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
1 -> 5  [arrowhead=odot, arrowtail=odot, dir=both];
2 -> 5  [arrowhead=odot, arrowtail=odot, dir=both];
}



In [93]:
Z = GetMB(g.graph, annotated_name)
Z

array(['flavor', 'aroma', 'size', 'juiciness'], dtype='<U9')

In [94]:
#print(*Eval_Factor_Discovery(Z), sep='\t')

possible_ancestors = get_possible_ancestors(g.graph, annotated_name)
print(possible_ancestors)
# print(*Eval_Factor_Confirmation(annotated_name, possible_ancestors, mode = 'ancestors'), sep='\t')


['texture', 'flavor', 'aroma', 'size', 'juiciness']


# Llama-2-70b



https://poe.com/Llama-2-70b

https://poe.com/s/HQ6znChHx9qERsCdBpiM


zeroshot https://poe.com/s/RkpCLbko6wLnqyx4ZZZn

In [230]:
this_bot_name = "Llama-2-70b-v1933" 
anno_model  = "Mixtral-8x7b-Groq" 
V = set([target_name])
Z = set()
annotation[this_bot_name] = {}

## iteration 1

In [165]:
print(propose_prompt_with_data_in_front(data))


You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data


## Group with 'Score' = -4

- This particular apple, while boasting a striking and consistent hue, falls short of expectations due to its unappealing small stature, an off-putting musty odor suggesting decay, and a flavor profile that tilts disappointingly toward sourness. The lack of juiciness further detracts from its desirability, making it a risky choice for suppliers and a potential loss-inducer in the market.
- Unfortunately, this batch of apples fails to meet quality standards. Their small size, extensive surface blemishes, and dry texture fall short of expectations. The off-putting musty scent suggests potential rot, and the sour taste overpowers any sweetness. Suppliers should beware; these apples could result in financial disappointment if offered to discerning customers.

## Group with 'Score' = -3

- Sadly, this apple batch underwhelms, presenting numer

In [166]:
result = """ 
<Factor Begin>

**Appearance**
- 1: Vibrant, uniform color
- 0: Otherwise; or not mentioned
- -1: Significant surface blemishes

**Aroma**
- 1: Strong, inviting aroma
- 0: Otherwise; or not mentioned
- -1: Off-putting musty or rotten aroma

**Flavor**
- 1: Well-balanced flavor profile
- 0: Otherwise; or not mentioned
- -1: Overwhelmingly sour or sweet taste

**Juiciness**
- 1: Abundant moisture
- 0: Otherwise; or not mentioned
- -1: Dry or mealy texture

"""
factor_list = result.split('<Factor Begin>')[-1].split('\n\n')

factor_list = check_factor_list(factor_list)
factor_list

['appearance\n- 1: Vibrant, uniform color\n- 0: Otherwise; or not mentioned\n- -1: Significant surface blemishes',
 'aroma\n- 1: Strong, inviting aroma\n- 0: Otherwise; or not mentioned\n- -1: Off-putting musty or rotten aroma',
 'flavor\n- 1: Well-balanced flavor profile\n- 0: Otherwise; or not mentioned\n- -1: Overwhelmingly sour or sweet taste',
 'juiciness\n- 1: Abundant moisture\n- 0: Otherwise; or not mentioned\n- -1: Dry or mealy texture']

### annotation

In [167]:
from utils import Logger
import time
anno_model  = "Mixtral-8x7b-Groq"  # "Llama-2-70b" #"Mistral-Medium"

sample_size = meta.shape[0]

for this_factor in factor_list:
    this_factor_name = get_factor_name(this_factor)

    if this_factor_name.lower() in '_'.join(annotation[this_bot_name].keys()).lower():
        continue
    
    print(this_factor_name)
    print(this_factor)

    logger = Logger(
        root = '/home/comp/cscxliu/COAT/AppleGastronome', 
        exp_name = f'Formal_exp_{exp_uid}_{benchmark_name}_anno_{this_factor_name}_for_{this_bot_name}_by_{anno_model}'
    )

    answers = []
    for idx in range(sample_size):
        print(f'\n ---- \n')
        anno_request = annotation_prompt(this_factor, meta.values[idx,-1].replace('\n', ' '))
        logger.straight_write('conversation', 'User: \n' + anno_request, mode='a')

        response = await chat(anno_request, bot_name=anno_model)
        logger.straight_write('conversation', f'{anno_model}: \n' + response, mode='a')

        this_value = get_value_from_responce(response)
        if this_value  in [None, '?']:
            this_value = 0

        answers.append(this_value)
        print(f'\n{idx+1}-th sample, factor "{this_factor_name}": {this_value}\n')
        time.sleep(0.2)
    annotation[this_bot_name][this_factor_name] = deepcopy(answers)

appearance
appearance
- 1: Vibrant, uniform color
- 0: Otherwise; or not mentioned
- -1: Significant surface blemishes

 ---- 


1-th sample, factor "appearance": 1


 ---- 


2-th sample, factor "appearance": 1


 ---- 


3-th sample, factor "appearance": 1


 ---- 


4-th sample, factor "appearance": 1


 ---- 


5-th sample, factor "appearance": 1


 ---- 


6-th sample, factor "appearance": 1


 ---- 


7-th sample, factor "appearance": 1


 ---- 


8-th sample, factor "appearance": -1


 ---- 


9-th sample, factor "appearance": 1


 ---- 


10-th sample, factor "appearance": 0


 ---- 


11-th sample, factor "appearance": 1


 ---- 


12-th sample, factor "appearance": -1


 ---- 


13-th sample, factor "appearance": -1


 ---- 


14-th sample, factor "appearance": -1


 ---- 


15-th sample, factor "appearance": -1


 ---- 


16-th sample, factor "appearance": 1


 ---- 


17-th sample, factor "appearance": -1


 ---- 


18-th sample, factor "appearance": 1


 ---- 


19-th samp

### confirmation

In [231]:
iteration = 1
print(f'e_{exp_uid}_{this_bot_name}_iter{iteration}')
file_path = f"/home/comp/cscxliu/COAT/AppleGastronome/data/Apple_Gastronome_AG7_v20240201_data_interface_{exp_uid}_{this_bot_name}_iter{iteration}.xlsx"

e_2024_02_01_1240_hk_Mixtral-8x7b-Groq_Llama-2-70b-v1933_iter1


In [169]:
# !!!
assert not os.path.exists(file_path)
annotation[this_bot_name][target_name] = meta[target_name].values
data_interface = pd.DataFrame(annotation[this_bot_name])
all_factors = list(data_interface.columns)

data_interface.to_excel(file_path, index=False) 

In [232]:

data_interface = pd.read_excel(file_path)
all_factors = list(data_interface.columns)

for f in all_factors:
    annotation[this_bot_name][f] = deepcopy(data_interface[f].to_list())

In [233]:
annotated_name = [target_name]+[f for idx, f in enumerate(all_factors) if (f != target_name)]
annoted_values = data_interface[annotated_name].values

ci_test = CIT(annoted_values)
alpha = 0.05

Sep = [idx for idx, f in enumerate(annotated_name) if (f in Z)and((f != target_name))]

valid_factors = [f for idx, f in enumerate(annotated_name) if (f not in V)and(ci_test(0,idx,Sep)<alpha)]
valid_factors

['aroma', 'flavor', 'juiciness']

In [234]:
print(V==V.union(valid_factors))
V=V.union(valid_factors)
V

False


{'aroma', 'flavor', 'juiciness', 'score'}

In [235]:
factors = [ f for f in V if f!= target_name]
annotated_name = [target_name]+factors
annoted_values = data_interface[annotated_name].values

# Causal Discovery (FCI)
g, edges = fci(annoted_values, alpha = 0.05, verbose=False)

# visualization
pdy = GraphUtils.to_pydot(g, labels=annotated_name)
print(annoted_values.std(0))
print(pdy.to_string())

Depth=0, working on node 3: 100%|██████████| 4/4 [00:00<00:00, 761.49it/s]

X1 --> X3
[1.82756669 0.89861004 0.77459667 0.92282176]
digraph  {
dpi=200;
fontsize=18;
0 [label=X1];
0 [label=score];
1 [label=X2];
1 [label=aroma];
2 [label=X3];
2 [label=flavor];
3 [label=X4];
3 [label=juiciness];
1 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
0 -> 2  [arrowhead=normal, arrowtail=none, dir=both];
3 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
}



In [236]:
Z = GetMB(g.graph, annotated_name)
Z

array(['score', 'aroma', 'flavor', 'juiciness'], dtype='<U9')

In [237]:
possible_ancestors = get_possible_ancestors(g.graph, annotated_name)
print(possible_ancestors)

['aroma', 'juiciness']


## iteration 2

### feedback

In [176]:
focus_factor = 'score'
focus_label = data_interface[focus_factor].values[:100]
background_factor = [f for f in list(Z) if f not in [focus_factor]]
cond_vectors = data_interface[background_factor].values[:100,:]

g = KMeans(n_clusters=min(len(factors), 4), random_state=0, n_init="auto").fit(cond_vectors).labels_

entropy_values = []
set_g = list(set(g))
for g_idx in set_g:
    qk = np.unique(focus_label[g==g_idx], return_counts=True)[1]
    n = qk.sum()
    qk = qk/ n
    this_entropy_values = stats.entropy(qk)
    if n < len(qk) * 3:
        this_entropy_values = -1
    entropy_values.append(this_entropy_values)
    print(g_idx, entropy_values[-1],n, qk)

g_chosen = set_g[np.argmax(entropy_values)]
print(f'choose g=={g_chosen}')


0 1.727446999545687 39 [0.02564103 0.05128205 0.20512821 0.30769231 0.15384615 0.17948718
 0.07692308]
1 1.4824259527039583 25 [0.08 0.08 0.48 0.08 0.2  0.08]
2 1.7202834815415742 36 [0.05555556 0.16666667 0.22222222 0.13888889 0.22222222 0.19444444]
choose g==0


In [177]:
example = ''

chosen_idx = g==g_chosen
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,-1]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example, [f  for f in data_interface.columns if f not in [target_name]]))



You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'score' = 0

---

 This apple delights with its striking, even hue, perfect for those favoring smaller fruit. Its potent aroma is an olfactory promise of the zestful taste it delivers, leaning more towards tartness than sugary notes. The juicy burst that accompanies each bite makes for an exhilaratingly crisp snacking experience, albeit perhaps a bit puckering for sweet-toothed consumers.

---

 This apple, with its vibrant, even hue and invitingly strong aroma, appeals to the senses. However, its sour profile over sweetness may limit its market appeal. Despite its abundant juiciness offering refreshing moisture, this characteristic divergence raises concerns for suppliers, suggesting it may not meet profitability expectations and could risk financial shortfalls.

---

 This compact apple surprises with a delightful sweetness that overshadows its tar

In [178]:
result = """ 
<Factor Begin>

**Appearance**
- 1: Uniform color, vibrant, and attractive.
- 0: Otherwise; or not mentioned.
- -1: Spotted, browned, or unattractive.

**Aroma**
- 1: Strong, inviting, and fragrant.
- 0: Otherwise; or not mentioned.
- -1: Weak, unpleasant, or sour.

**Flavor**
- 1: Sweet, balanced, and refreshing.
- 0: Otherwise; or not mentioned.
- -1: Sour, unbalanced, or unpleasant.

**Juiciness**
- 1: High, refreshing, and moist.
- 0: Otherwise; or not mentioned.
- -1: Low, dry, or watery.

**Texture**
- 1: Crunchy, firm, and satisfying.
- 0: Otherwise; or not mentioned.
- -1: Soft, mushy, or unpleasant.

**Acidity**
- 1: High, tangy, and zesty.
- 0: Otherwise; or not mentioned.
- -1: Low, flat, or uninteresting.

**Sweetness**
- 1: High, rich, and indulgent.
- 0: Otherwise; or not mentioned.
- -1: Low, tart, or unpleasant.

**Browning**
- 1: Minimal, inconspicuous, or none.
- 0: Otherwise; or not mentioned.
- -1: Prominent, noticeable, or extensive.

"""
factor_list = result.split('<Factor Begin>')[-1].split('\n\n')

factor_list = check_factor_list(factor_list)
factor_list

['appearance\n- 1: Uniform color, vibrant, and attractive.\n- 0: Otherwise; or not mentioned.\n- -1: Spotted, browned, or unattractive.',
 'aroma\n- 1: Strong, inviting, and fragrant.\n- 0: Otherwise; or not mentioned.\n- -1: Weak, unpleasant, or sour.',
 'flavor\n- 1: Sweet, balanced, and refreshing.\n- 0: Otherwise; or not mentioned.\n- -1: Sour, unbalanced, or unpleasant.',
 'juiciness\n- 1: High, refreshing, and moist.\n- 0: Otherwise; or not mentioned.\n- -1: Low, dry, or watery.',
 'texture\n- 1: Crunchy, firm, and satisfying.\n- 0: Otherwise; or not mentioned.\n- -1: Soft, mushy, or unpleasant.',
 'acidity\n- 1: High, tangy, and zesty.\n- 0: Otherwise; or not mentioned.\n- -1: Low, flat, or uninteresting.',
 'sweetness\n- 1: High, rich, and indulgent.\n- 0: Otherwise; or not mentioned.\n- -1: Low, tart, or unpleasant.',
 'browning\n- 1: Minimal, inconspicuous, or none.\n- 0: Otherwise; or not mentioned.\n- -1: Prominent, noticeable, or extensive.']

### annotation

In [179]:
from utils import Logger
import time
anno_model  = "Mixtral-8x7b-Groq" # "Llama-2-70b"  # "Llama-2-70b" #"Mistral-Medium"

sample_size = meta.shape[0]

for this_factor in factor_list:
    this_factor_name = get_factor_name(this_factor)
    
    if this_factor_name.lower() in '_'.join(annotation[this_bot_name].keys()).lower():
        print(f'skip {this_factor_name}')
        continue
    print(this_factor_name)
    print(this_factor)

    logger = Logger(
        root = '/home/comp/cscxliu/COAT/AppleGastronome', 
        exp_name = f'Formal_exp_{exp_uid}_{benchmark_name}_anno_{this_factor_name}_for_{this_bot_name}_by_{anno_model}'
    )

    answers = []
    for idx in range(sample_size):
        print(f'\n ---- \n')
        anno_request = annotation_prompt(this_factor, meta.values[idx,-1].replace('\n', ' '))
        logger.straight_write('conversation', 'User: \n' + anno_request, mode='a')

        response = await chat(anno_request, bot_name=anno_model)
        logger.straight_write('conversation', f'{anno_model}: \n' + response, mode='a')

        this_value = get_value_from_responce(response)
        if this_value  in [None, '?']:
            this_value = 0

        answers.append(this_value)
        print(f'\n{idx+1}-th sample, factor "{this_factor_name}": {this_value}\n')
        time.sleep(0.2)
    annotation[this_bot_name][this_factor_name] = deepcopy(answers)

skip appearance
skip aroma
skip flavor
skip juiciness
texture
texture
- 1: Crunchy, firm, and satisfying.
- 0: Otherwise; or not mentioned.
- -1: Soft, mushy, or unpleasant.

 ---- 


1-th sample, factor "texture": 0


 ---- 


2-th sample, factor "texture": 0


 ---- 


3-th sample, factor "texture": 0


 ---- 


4-th sample, factor "texture": -1


 ---- 


5-th sample, factor "texture": -1


 ---- 


6-th sample, factor "texture": 1


 ---- 


7-th sample, factor "texture": 1


 ---- 


8-th sample, factor "texture": 1


 ---- 


9-th sample, factor "texture": 0


 ---- 


10-th sample, factor "texture": 0


 ---- 


11-th sample, factor "texture": 0


 ---- 


12-th sample, factor "texture": 0


 ---- 


13-th sample, factor "texture": 0


 ---- 


14-th sample, factor "texture": -1


 ---- 


15-th sample, factor "texture": -1


 ---- 


16-th sample, factor "texture": -1


 ---- 


17-th sample, factor "texture": 0


 ---- 


18-th sample, factor "texture": -1


 ---- 


19-th sam

### confirmation

In [238]:
iteration = 2
print(exp_uid, this_bot_name, iteration)
file_path = f"/home/comp/cscxliu/COAT/AppleGastronome/data/Apple_Gastronome_AG7_v20240201_data_interface_{exp_uid}_{this_bot_name}_iter{iteration}.xlsx"


2024_02_01_1240_hk_Mixtral-8x7b-Groq Llama-2-70b-v1933 2


In [181]:
# !!!
assert not os.path.exists(file_path)
annotation[this_bot_name][target_name] = meta[target_name].values
data_interface = pd.DataFrame(annotation[this_bot_name])
all_factors = list(data_interface.columns)

data_interface.to_excel(file_path, index=False) 

In [239]:

data_interface = pd.read_excel(file_path)
all_factors = list(data_interface.columns)

for f in all_factors:
    annotation[this_bot_name][f] = deepcopy(data_interface[f].to_list())

In [240]:
annotated_name = [target_name]+[f for idx, f in enumerate(all_factors) if (f != target_name)]
annoted_values = data_interface[annotated_name].values

ci_test = CIT(annoted_values)
alpha = 0.05
Sep = [idx for idx, f in enumerate(annotated_name) if (f in Z)and((f != target_name))]

valid_factors = [f for idx, f in enumerate(annotated_name) if (f not in V)and(ci_test(0,idx,Sep)<alpha)]
valid_factors

['texture', 'acidity', 'sweetness']

In [241]:
print(V==V.union(valid_factors))
V=V.union(valid_factors)
V

False


{'acidity', 'aroma', 'flavor', 'juiciness', 'score', 'sweetness', 'texture'}

In [242]:
factors = [ f for f in V if f!= target_name]
annotated_name = [target_name]+factors
annoted_values = data_interface[annotated_name].values

# Causal Discovery (FCI)
g, edges = fci(annoted_values, alpha = 0.05, verbose=False)

# visualization
pdy = GraphUtils.to_pydot(g, labels=annotated_name)
print(annoted_values.std(0))
print(pdy.to_string())

Depth=0, working on node 6: 100%|██████████| 7/7 [00:00<00:00, 619.69it/s]

X3 --> X1
X7 --> X1
X3 --> X6
[1.82756669 0.80616376 0.8330066  0.89861004 0.59866518 0.77459667
 0.92282176]
digraph  {
dpi=200;
fontsize=18;
0 [label=X1];
0 [label=score];
1 [label=X2];
1 [label=acidity];
2 [label=X3];
2 [label=sweetness];
3 [label=X4];
3 [label=aroma];
4 [label=X5];
4 [label=texture];
5 [label=X6];
5 [label=flavor];
6 [label=X7];
6 [label=juiciness];
2 -> 0  [arrowhead=normal, arrowtail=none, dir=both];
3 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
5 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
6 -> 0  [arrowhead=normal, arrowtail=none, dir=both];
1 -> 2  [arrowhead=normal, arrowtail=odot, dir=both];
2 -> 5  [arrowhead=normal, arrowtail=none, dir=both];
2 -> 6  [arrowhead=normal, arrowtail=normal, dir=both];
3 -> 4  [arrowhead=normal, arrowtail=odot, dir=both];
4 -> 6  [arrowhead=normal, arrowtail=normal, dir=both];
}



In [243]:
Z = GetMB(g.graph, annotated_name)
Z

array(['sweetness', 'aroma', 'flavor', 'juiciness'], dtype='<U9')

In [244]:
possible_ancestors = get_possible_ancestors(g.graph, annotated_name)
print(possible_ancestors)

['acidity', 'sweetness', 'aroma', 'flavor', 'juiciness']


## iteration 3

### feedback

In [245]:
focus_factor = 'score'
focus_label = data_interface[focus_factor].values[:100]
background_factor = [f for f in list(Z) if f not in [focus_factor]]
cond_vectors = data_interface[background_factor].values[:100,:]

g = KMeans(n_clusters=min(len(factors), 4), random_state=0, n_init="auto").fit(cond_vectors).labels_

entropy_values = []
set_g = list(set(g))
for g_idx in set_g:
    qk = np.unique(focus_label[g==g_idx], return_counts=True)[1]
    n = qk.sum()
    qk = qk/ n
    this_entropy_values = stats.entropy(qk)
    if n < len(qk) * 3:
        this_entropy_values = -1
    entropy_values.append(this_entropy_values)
    print(g_idx, entropy_values[-1],n, qk)

g_chosen = set_g[np.argmax(entropy_values)]
print(f'choose g=={g_chosen}')


0 1.3180238017106036 17 [0.05882353 0.11764706 0.35294118 0.41176471 0.05882353]
1 1.4273606178598333 26 [0.07692308 0.07692308 0.46153846 0.03846154 0.26923077 0.07692308]
2 1.727694669998594 32 [0.0625  0.1875  0.25    0.15625 0.1875  0.15625]
3 1.5765247610320796 25 [0.2  0.2  0.2  0.28 0.12]
choose g==2


In [246]:
example = ''

chosen_idx = g==g_chosen
for f_v in set(focus_label[chosen_idx]):
    example +=  f'\n\n## group with \'{focus_factor}\' = {f_v}'
    these_indeces = np.arange(len(focus_label))[(focus_label == f_v)&(chosen_idx)]
    if len(these_indeces) > 3:
        these_indeces = np.random.choice(these_indeces, 3, replace=False)
    for each_idx in these_indeces:
        this_review = meta.values[each_idx,-1]
        pieces = this_review.split('\n\n')[:2]
        del_idx = []
        for idx, piece in enumerate(pieces):
            if 'pattern' in piece.lower():
                del_idx += [idx, idx+1]
        this_review = '\n\n'.join([pieces[idx] for idx in range(len(pieces)) if idx not in del_idx])
        example +=  f'\n\n---\n\n{this_review}'
print(propose_prompt_with_data_in_front(example, [f  for f in data_interface.columns if f not in [target_name]]))



You are an excellently helpful AI assistant for analyzing, abstracting, and processing data. Now try your best.

# Data



## group with 'score' = 0

---

 With a vibrant, uniform color, this pint-sized apple exudes a strong aroma that hints at its robust character. Despite a flavor leaning more towards tartness than sweetness and a texture that is somewhat parched, its unique qualities suggest it holds considerable promise in the marketplace and merits greater acclaim among apple aficionados.

---

 The apple in question disappoints with a plethora of unsightly spots and an uninviting browning of the skin. Despite a robust aroma hinting at freshness, the fruit's taste leans unpleasantly sour, troubling the palate more than it pleases. A disappointing lack of juiciness further compounds the matter, presenting a dry texture. Its quality is unlikely to meet commercial expectations, risking financial loss for suppliers and dissatisfaction for customers.

---

 Despite its inviting strong

In [247]:
result = """ 
<Factor Begin>

- 1: Sweetness
	* 1: Strongly sweet
	* 0: Balanced or slightly sweet
	* -1: Tart or sour

- 2: Size
	* 1: Large
	* 0: Medium
	* -1: Small

- 3: Aroma
	* 1: Potent and inviting
	* 0: Moderate or neutral
	* -1: Weak or unpleasant

- 4: Juiciness
	* 1: Very juicy
	* 0: Moderately juicy
	* -1: Dry or not juicy

- 5: Balance
	* 1: Well-balanced
	* 0: Slightly unbalanced
	* -1: Significantly unbalanced

"""
factor_list = result.split('<Factor Begin>')[-1].split('\n\n')

factor_list = check_factor_list(factor_list)
factor_list

['sweetness\n\t* 1: Strongly sweet\n\t* 0: Balanced or slightly sweet\n\t* -1: Tart or sour',
 'size\n\t* 1: Large\n\t* 0: Medium\n\t* -1: Small',
 'aroma\n\t* 1: Potent and inviting\n\t* 0: Moderate or neutral\n\t* -1: Weak or unpleasant',
 'juiciness\n\t* 1: Very juicy\n\t* 0: Moderately juicy\n\t* -1: Dry or not juicy',
 'balance\n\t* 1: Well-balanced\n\t* 0: Slightly unbalanced\n\t* -1: Significantly unbalanced']

### annotation

In [248]:
from utils import Logger
import time
anno_model  = "Mixtral-8x7b-Groq" # "Llama-2-70b"  # "Llama-2-70b" #"Mistral-Medium"

sample_size = meta.shape[0]

for this_factor in factor_list:
    this_factor_name = get_factor_name(this_factor)
    
    if this_factor_name.lower() in '_'.join(annotation[this_bot_name].keys()).lower():
        print(f'skip {this_factor_name}')
        continue
    print(this_factor_name)
    print(this_factor)

    logger = Logger(
        root = '/home/comp/cscxliu/COAT/AppleGastronome', 
        exp_name = f'Formal_exp_{exp_uid}_{benchmark_name}_anno_{this_factor_name}_for_{this_bot_name}_by_{anno_model}'
    )

    answers = []
    for idx in range(sample_size):
        print(f'\n ---- \n')
        anno_request = annotation_prompt(this_factor, meta.values[idx,-1].replace('\n', ' '))
        logger.straight_write('conversation', 'User: \n' + anno_request, mode='a')

        response = await chat(anno_request, bot_name=anno_model)
        logger.straight_write('conversation', f'{anno_model}: \n' + response, mode='a')

        this_value = get_value_from_responce(response)
        if this_value  in [None, '?']:
            this_value = 0

        answers.append(this_value)
        print(f'\n{idx+1}-th sample, factor "{this_factor_name}": {this_value}\n')
        time.sleep(0.2)
    annotation[this_bot_name][this_factor_name] = deepcopy(answers)

skip sweetness
size
size
	* 1: Large
	* 0: Medium
	* -1: Small

 ---- 




1-th sample, factor "size": 1


 ---- 


2-th sample, factor "size": 1


 ---- 


3-th sample, factor "size": 1


 ---- 


4-th sample, factor "size": -1


 ---- 


5-th sample, factor "size": -1


 ---- 


6-th sample, factor "size": 0


 ---- 


7-th sample, factor "size": 1


 ---- 


8-th sample, factor "size": -1


 ---- 


9-th sample, factor "size": -1


 ---- 


10-th sample, factor "size": -1


 ---- 


11-th sample, factor "size": 1


 ---- 


12-th sample, factor "size": -1


 ---- 


13-th sample, factor "size": -1


 ---- 


14-th sample, factor "size": 0


 ---- 


15-th sample, factor "size": -1


 ---- 


16-th sample, factor "size": 0


 ---- 


17-th sample, factor "size": -1


 ---- 


18-th sample, factor "size": -1


 ---- 


19-th sample, factor "size": -1


 ---- 


20-th sample, factor "size": -1


 ---- 


21-th sample, factor "size": 1


 ---- 


22-th sample, factor "size": 1


 ---- 


23-th sample, factor "size": 1


 ---- 


24-th sample, factor "size": 1

### confirmation

In [225]:
iteration = 3
print(exp_uid, this_bot_name, iteration)
file_path = f"/home/comp/cscxliu/COAT/AppleGastronome/data/Apple_Gastronome_AG7_v20240201_data_interface_{exp_uid}_{this_bot_name}_iter{iteration}.xlsx"


2024_02_01_1240_hk_Mixtral-8x7b-Groq Llama-2-70b-v1933 3


In [226]:
# !!!
assert not os.path.exists(file_path)
annotation[this_bot_name][target_name] = meta[target_name].values
data_interface = pd.DataFrame(annotation[this_bot_name])
all_factors = list(data_interface.columns)

data_interface.to_excel(file_path, index=False) 

In [227]:

data_interface = pd.read_excel(file_path)
all_factors = list(data_interface.columns)

for f in all_factors:
    annotation[this_bot_name][f] = deepcopy(data_interface[f].to_list())

In [229]:
annotated_name = [target_name]+[f for idx, f in enumerate(all_factors) if (f != target_name)]
annoted_values = data_interface[annotated_name].values

ci_test = CIT(annoted_values)
alpha = 0.05

Sep = [idx for idx, f in enumerate(annotated_name) if (f in Z)and((f != target_name))]

valid_factors = [f for idx, f in enumerate(annotated_name) if (f not in V)and(ci_test(0,idx,Sep)<alpha)]
valid_factors

[]

In [249]:
print(V==V.union(valid_factors))
V=V.union(valid_factors)
V

True


{'acidity', 'aroma', 'flavor', 'juiciness', 'score', 'sweetness', 'texture'}

In [253]:
factors = [ f for f in V if f!= target_name]
annotated_name = [target_name]+factors
annoted_values = data_interface[annotated_name].values

# Causal Discovery (FCI)
g, edges = fci(annoted_values, alpha = 0.05, verbose=False)

# visualization
pdy = GraphUtils.to_pydot(g, labels=annotated_name)
print(annoted_values.std(0))
print(pdy.to_string())

Depth=0, working on node 6: 100%|██████████| 7/7 [00:00<00:00, 661.50it/s]

X3 --> X1
X6 --> X1
X6 --> X5
[1.82756669 0.89861004 0.92282176 0.59866518 0.77459667 0.8330066
 0.80616376]
digraph  {
dpi=200;
fontsize=18;
0 [label=X1];
0 [label=score];
1 [label=X2];
1 [label=aroma];
2 [label=X3];
2 [label=juiciness];
3 [label=X4];
3 [label=texture];
4 [label=X5];
4 [label=flavor];
5 [label=X6];
5 [label=sweetness];
6 [label=X7];
6 [label=acidity];
1 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
2 -> 0  [arrowhead=normal, arrowtail=none, dir=both];
4 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
5 -> 0  [arrowhead=normal, arrowtail=none, dir=both];
1 -> 3  [arrowhead=normal, arrowtail=odot, dir=both];
2 -> 3  [arrowhead=normal, arrowtail=normal, dir=both];
2 -> 5  [arrowhead=normal, arrowtail=normal, dir=both];
5 -> 4  [arrowhead=normal, arrowtail=none, dir=both];
6 -> 5  [arrowhead=normal, arrowtail=odot, dir=both];
}



In [254]:
Z = GetMB(g.graph, annotated_name)
Z

array(['aroma', 'juiciness', 'flavor', 'sweetness'], dtype='<U9')

In [255]:
possible_ancestors = get_possible_ancestors(g.graph, annotated_name)
print(possible_ancestors)

['aroma', 'juiciness', 'flavor', 'sweetness', 'acidity']
